In [1]:
!ls data/han/AWS/

남이섬_2016.xlsx  남이섬_2020.xlsx  청평_2019.xlsx  화천_2018.xlsx
남이섬_2017.xlsx  청평_2016.xlsx    청평_2020.xlsx  화천_2019.xlsx
남이섬_2018.xlsx  청평_2017.xlsx    화천_2016.xlsx  화천_2020.xlsx
남이섬_2019.xlsx  청평_2018.xlsx    화천_2017.xlsx


In [2]:
!pwd

/home/cjinw/work/water-quality/rnn


In [3]:
%matplotlib widget
import pandas as pd
import numpy as np
from glob import glob
import os
import datetime
import matplotlib.pyplot as plt
    
from tensorflow.keras.layers import Input, Concatenate, Dot, Add, ReLU, Activation
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from tensorflow import keras

In [4]:
# for Korean in plot
import matplotlib
import matplotlib.font_manager as fm
fm.get_fontconfig_fonts()
font_location = '/usr/share/fonts/truetype/nanum/NanumGothicCoding.ttf'
#font_location = '/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc'
# font_location = 'C:/Windows/Fonts/NanumGothic.ttf' # For Windows
fprop = fm.FontProperties(fname=font_location)

In [5]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

In [6]:

# folder = 'data/han/자동'
# folder = 'data/han/수질'
# folder = 'data/han/AWS'
# folder = 'data/nakdong/자동'
# folder = 'data/gum/자동'
folder = 'data/yeongsan1/자동'

# ## 한강
# ## 자동
# file_names = [['가평_2016.xlsx','가평_2017.xlsx','가평_2018.xlsx', '가평_2019.xlsx'], 
#               ['의암호_2016.xlsx','의암호_2017.xlsx','의암호_2018.xlsx', '의암호_2019.xlsx'],
#               ['서상_2016.xlsx','서상_2017.xlsx','서상_2018.xlsx', '서상_2019.xlsx']]

## 수질
# file_names = [['가평천3_2016.xlsx','가평천3_2017.xlsx','가평천3_2018.xlsx', '가평천3_2019.xlsx'],
#               ['대성리_2016.xlsx','대성리_2017.xlsx','대성리_2018.xlsx', '대성리_2019.xlsx'],
#               ['청평댐3_2016.xlsx','청평댐3_2017.xlsx','청평댐3_2018.xlsx', '청평댐3_2019.xlsx'],
#               ['춘천댐1_2016.xlsx','춘천댐1_2017.xlsx','춘천댐1_2018.xlsx', '춘천댐1_2019.xlsx'],
#               ['대성리_2016.xlsx','대성리_2017.xlsx','대성리_2018.xlsx', '대성리_2019.xlsx'],
#               ['의암_2016.xlsx','의암_2017.xlsx','의암_2018.xlsx', '의암_2019.xlsx'],
#               ['춘성교_2016.xlsx','춘성교_2017.xlsx','춘성교_2018.xlsx', '춘성교_2019.xlsx'],
#               ['남이섬_2016.xlsx','남이섬_2017.xlsx','남이섬_2018.xlsx', '남이섬_2019.xlsx'],
#               ['청평댐3_2016.xlsx','청평댐3_2017.xlsx','청평댐3_2018.xlsx', '청평댐3_2019.xlsx'],
#               ['춘천_2016.xlsx','춘천_2017.xlsx','춘천_2018.xlsx', '춘천_2019.xlsx']
#              ]


## AWS
# file_names = [['남이섬_2016.xlsx','남이섬_2017.xlsx','남이섬_2018.xlsx', '남이섬_2018.xlsx'],
#               ['청평_2016.xlsx', '청평_2017.xlsx', '청평_2018.xlsx', '청평_2019.xlsx'],
#               ['화천_2016.xlsx', '화천_2017.xlsx', '화천_2018.xlsx', '화천_2019.xlsx']]
 

    
## 낙동강
# #자동
# file_names = [['도개_2016.xlsx','도개_2017.xlsx','도개_2018.xlsx', '도개_2019.xlsx'],
#               ['신암_2016.xlsx','신암_2017.xlsx','신암_2018.xlsx', '신암_2019.xlsx'],
#               ['회상_2016.xlsx','회상_2017.xlsx','회상_2018.xlsx', '회상_2019.xlsx']]


## 금강
#자동
# file_names = [['현도_2016.xlsx','현도_2017.xlsx','현도_2018.xlsx', '현도_2019.xlsx'],
#               ['대청호_2016.xlsx','대청호_2017.xlsx','대청호_2018.xlsx', '대청호_2019.xlsx'],
#               ['옥천천_2016.xlsx','옥천천_2017.xlsx','옥천천_2018.xlsx', '옥천천_2019.xlsx'],
#               ['장계_2016.xlsx','장계_2017.xlsx','장계_2018.xlsx', '장계_2019.xlsx']]
             

    
    
## 영산강
#자동
file_names = [['나주_2016.xlsx','나주_2017.xlsx','나주_2018.xlsx', '나주_2019.xlsx'],
              ['지석천_2016.xlsx','지석천_2017.xlsx','지석천_2018.xlsx', '지석천_2019.xlsx'],
              ['용봉_2016.xlsx','용봉_2017.xlsx','용봉_2018.xlsx', '용봉_2019.xlsx'],
#               ['서창교_2016.xlsx','서창교_2017.xlsx','서창교_2018.xlsx', '서창교_2019.xlsx'],
              ['우치_2016.xlsx','우치_2017.xlsx','우치_2018.xlsx', '우치_2019.xlsx']]
              

In [7]:
target_col = '용존산소'
# 각각의 강 마다, 측정소 별로 column을 맞춰야함
# 0: 자동, 1: 수질, 2: AWS(ASOS), 
han = [':,2:', ':,28:45', ':,26:29']
nakdong = [':,2:', ':,2:12']
gum = [':,2:'] #, ':,28:45', ':,26:29']
# gain_han = ['save/han/1', 'save/han/2', 'save/han/3']

# interpolation_option = True
interpolation_option = False
# gain_save_path = gain_han[2]

iloc_val = gum[0]
iloc_val

':,2:'

In [8]:
def make_columns(df):
    column_list = ['측정날짜', '측정소명', '수온', '수소이온농도','전기전도도', '용존산소', '총유기탄소', '총질소', '총인', '클로로필-a']
    list_df = pd.DataFrame(columns=column_list)
    list_df
    df = df.drop(columns=df.columns.difference(column_list))
    new_column = list_df.columns.difference(df.columns)
#     print(new_column)
    if not new_column.empty :
        print("Make_columns")
        for i in range(new_column.shape[0]):
            df[new_column[i]] = pd.Series()
#     print('columns')
#     print(df.columns)
    return df


In [9]:
def make_timeseries(df, interpolate=None):
    print(df.shape)
    date_col = df.columns[0]
    df[date_col] = pd.to_datetime(df[date_col])
    df = df[df[date_col].notna()]

    year = pd.DatetimeIndex(df[date_col]).year.astype(np.int64)

    start = str(year[0]) + "-01-01 00:00"
#     start

    end = str(year[0]) + "-12-31 23:00"
#     end

    time_series = pd.date_range(start = start, end = end, freq='H')
    time_series
    time_series = pd.DataFrame(time_series)
    time_series.columns = [date_col]   
    time_series

    df = pd.concat([time_series, df], axis=0)
    df = df.drop_duplicates([date_col], keep="last")
    df = df.sort_values([date_col], axis=0)
    
    if interpolate :
        print('interpolation')
        df = df.interpolate(metod='spline', order=2, limit_direction='both')
        
    print(df.shape)
    return df

In [10]:
def make_dataframe(file_names, iloc_val, interpolate=None):
    day = 24*60*60
    year = (365.2425)*day

    df_full = []
    df = []
    
    for loc in range(len(file_names)):
    
        df_loc = []
        for y in range(len(file_names[loc])):
            path = os.path.join(folder, file_names[loc][y])
            print(file_names[loc][y])
            df_tmp = make_timeseries(pd.read_excel(path), interpolate=interpolate)
#             df_tmp = pd.read_excel(path).dropna(axis='columns', how = 'all')
            df_tmp = make_columns(df_tmp)
            
    #         df_tmp = pd.read_excel(path).dropna(axis='columns', how = 'all')
    #         print(df_tmp.head) 
            df_loc.append(df_tmp)
#             df_loc.append(pd.read_excel(path))
        df_full.append(pd.concat(df_loc))
    #     df.append(df_full[loc].iloc[:,2:11])
        df.append(eval(f"df_full[loc].iloc[{iloc_val}]"))
    #     df.append(df_full[loc].iloc[:, [2,3,4,5,6,7,10]])
    #     df.append(df_full[loc].iloc[:, 2:11])
        date_time = pd.to_datetime(df_full[loc].iloc[:, 0], format='%Y.%m.%d %H:%M', utc=True)
        timestamp_s = date_time.map(datetime.datetime.timestamp)
        df[loc]['Day sin'] = np.sin(timestamp_s * (2 * np.pi / day))
        df[loc]['Day cos'] = np.cos(timestamp_s * (2 * np.pi / day))
        df[loc]['Year sin'] = np.sin(timestamp_s * (2 * np.pi / year))
        df[loc]['Year cos'] = np.cos(timestamp_s * (2 * np.pi / year))
        df[loc] = df[loc].reset_index(drop=True)
        
#         print(df[loc].shape[0])
#         rows = df[loc].shape[0]
#         percentage = round(rows * 0.7)
#         df[loc] = df[loc].dropna(axis='columns',  thresh=percentage)
        

    
    return df

In [11]:
def normalize(df):
    # normalize data
    df_all = pd.concat(df)
    
    train_mean = df_all.mean()
    train_std = df_all.std()
    for i in range(len(file_names)):
        df[i] = (df[i]-train_mean)/train_std
    
    
    return df_all, train_mean, train_std

In [12]:
def min_max_normalize(df):
    # normalize data
    df_all = pd.concat(df)
    
    train_min = df_all.min()
    train_max = df_all.max()
    for i in range(len(file_names)):
        df[i] = (df[i]-train_min)/(train_max - train_min)       
    
    
    return df_all, train_min, train_max

In [13]:
##for normal
# df = make_dataframe(file_names, iloc_val)
## for interpolation
df = make_dataframe(file_names, iloc_val, interpolate=interpolation_option)
df_all, train_mean, train_std = normalize(df)
# df_all, train_mean, train_std = min_max_normalize(df)

나주_2016.xlsx
(8636, 33)
(8784, 33)
나주_2017.xlsx
(8747, 33)
(8760, 33)
나주_2018.xlsx
(8760, 33)
(8760, 33)
나주_2019.xlsx
(8760, 33)
(8760, 33)
지석천_2016.xlsx
(8784, 10)
(8784, 10)
지석천_2017.xlsx
(8760, 10)
(8760, 10)
지석천_2018.xlsx
(8745, 10)
(8760, 10)
지석천_2019.xlsx
(8757, 10)
(8760, 10)
용봉_2016.xlsx
(8784, 32)
(8784, 32)
용봉_2017.xlsx
(8730, 32)
(8760, 32)
용봉_2018.xlsx
(8758, 32)
(8760, 32)
용봉_2019.xlsx
(8760, 32)
(8760, 32)
우치_2016.xlsx
(8784, 10)
(8784, 10)
우치_2017.xlsx
(8760, 10)
(8760, 10)
우치_2018.xlsx
(8747, 10)
(8760, 10)
우치_2019.xlsx
(8760, 10)
(8760, 10)


In [14]:
train_mean, train_std

(수온          1.754832e+01
 수소이온농도      7.404510e+00
 전기전도도       2.903623e+02
 용존산소        8.789810e+00
 총유기탄소       3.848232e+00
 총질소         3.556650e+00
 총인          9.112342e-02
 클로로필-a      3.692030e+01
 Day sin     6.037102e-15
 Day cos     2.747704e-15
 Year sin   -5.681266e-08
 Year cos    2.053380e-05
 dtype: float64,
 수온           8.616625
 수소이온농도       0.765725
 전기전도도       89.425099
 용존산소         3.681855
 총유기탄소        1.151644
 총질소          1.781291
 총인           0.047667
 클로로필-a      38.660971
 Day sin      0.707109
 Day cos      0.707109
 Year sin     0.707102
 Year cos     0.707117
 dtype: float64)

## 

In [15]:
# just for Create WidnowGenerator 
train_df = df[0]
val_df = df[0]
test_df = df[0]

In [16]:
class WindowGenerator():
  def __init__(self, input_width, label_width, shift,
               train_df=train_df, val_df=val_df, test_df=test_df,
            #train_df=None, val_df=None, test_df=None,
#                out_features = None,
               out_features = None, out_num_features = None,
               label_columns=None):
    # Store the raw data.
    self.train_df = train_df
    self.val_df = val_df
    self.test_df = test_df

    # Work out the label column indices.
    self.label_columns = label_columns
    if label_columns is not None:
      self.label_columns_indices = {name: i for i, name in
                                    enumerate(label_columns)}
    self.column_indices = {name: i for i, name in
                           enumerate(train_df.columns)}

    # Work out the window parameters.
    self.input_width = input_width
    self.label_width = label_width
    self.shift = shift

    self.total_window_size = input_width + shift

    self.input_slice = slice(0, input_width)
    self.input_indices = np.arange(self.total_window_size)[self.input_slice]

    self.label_start = self.total_window_size - self.label_width
    self.labels_slice = slice(self.label_start, None)
    self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

  def __repr__(self):
    return '\n'.join([
        f'Total window size: {self.total_window_size}',
        f'Input indices: {self.input_indices}',
        f'Label indices: {self.label_indices}',
        f'Label column name(s): {self.label_columns}'])

In [17]:
def split_window(self, features):
  inputs = features[:, self.input_slice, :]
  labels = features[:, self.labels_slice, :]
  if self.label_columns is not None:
    labels = tf.stack(
        [labels[:, :, self.column_indices[name]] for name in self.label_columns],
        axis=-1)

  # Slicing doesn't preserve static shape information, so set the shapes
  # manually. This way the `tf.data.Datasets` are easier to inspect.
  inputs.set_shape([None, self.input_width, None])
  labels.set_shape([None, self.label_width, None])

  return inputs, labels

WindowGenerator.split_window = split_window

In [18]:
def plot(self, model=None, plot_col='T (degC)', max_subplots=3):
  inputs, labels = self.example
  plt.figure(figsize=(10, 8))
  plot_col_index = self.column_indices[plot_col]
  max_n = min(max_subplots, len(inputs))
  for n in range(max_n):
    plt.subplot(3, 1, n+1)
    plt.ylabel(f'{plot_col} [normed]', fontproperties=fprop)
    plt.plot(self.input_indices, inputs[n, :, plot_col_index],
             label='Inputs', marker='.', zorder=-10)

    if self.label_columns:
      label_col_index = self.label_columns_indices.get(plot_col, None)
    else:
      label_col_index = plot_col_index

    if label_col_index is None:
      continue

    plt.scatter(self.label_indices, labels[n, :, label_col_index],
                edgecolors='k', label='Labels', c='#2ca02c', s=64)
    if model is not None:
      predictions = model(inputs)
      plt.scatter(self.label_indices, predictions[n, :, label_col_index],
                  marker='X', edgecolors='k', label='Predictions',
                  c='#ff7f0e', s=64)

    if n == 0:
      plt.legend()

  plt.xlabel('Time [h]')

WindowGenerator.plot = plot

In [19]:
@property
def train(self):
  return self.make_dataset(self.train_df)

@property
def val(self):
  return self.make_dataset(self.val_df)

@property
def test(self):
  return self.make_dataset(self.test_df)

@property
def example(self):
  """Get and cache an example batch of `inputs, labels` for plotting."""
  result = getattr(self, '_example', None)
  if result is None:
    # No example batch was found, so get one from the `.train` dataset
    result = next(iter(self.train))
    # And cache it for next time
    self._example = result
  return result
@property
def example2(self):
  """Get and cache an example batch of `inputs, labels` for plotting."""
  result = getattr(self, '_example2', None)
  if result is None:
    # No example batch was found, so get one from the `.train` dataset
    result = next(iter(self.val))
    # And cache it for next time
    self._example = result
  return result
@property
def example3(self):
  """Get and cache an example batch of `inputs, labels` for plotting."""
  result = getattr(self, '_example3', None)
  if result is None:
    # No example batch was found, so get one from the `.train` dataset
    result = next(iter(self.test))
    # And cache it for next time
    self._example = result
  return result


WindowGenerator.train = train
WindowGenerator.val = val
WindowGenerator.test = test
WindowGenerator.example = example
WindowGenerator.example2 = example2
WindowGenerator.example3 = example3

In [20]:
def sample_batch_index(total, batch_size):
    '''Sample index of the mini-batch.

    Args:
        - total: total number of samples
        - batch_size: batch size

    Returns:
        - batch_idx: batch index
    '''
    total_idx = np.random.permutation(total)
    batch_idx = total_idx[:batch_size]
    return batch_idx

In [21]:
def binary_sampler(p, shape):
  '''Sample binary random variables.
  
  Args:
    - p: probability of 1
    - shape: matrix shape
    
  Returns:
    - binary_random_matrix: generated binary random matrix.
  '''
  unif_random_matrix = np.random.uniform(0., 1., size = shape)
  binary_random_matrix = 1*(unif_random_matrix < p)
  return binary_random_matrix

In [22]:
def uniform_sampler(low, high, shape):
  '''Sample uniform random variables.
  
  Args:
    - low: low limit
    - high: high limit
    - rows: the number of rows
    - cols: the number of columns
    
  Returns:
    - uniform_random_matrix: generated uniform random matrix.
  '''
  return np.random.uniform(low, high, size = shape)

In [23]:
class MissData(object):
    def __init__(self, load_dir=None):
        if load_dir:
            self.missarr = np.load(os.path.join(load_dir, 'miss.npy'))
            self.idxarr = np.load(os.path.join(load_dir, 'idx.npy'))
            
    def make_missdata(self, data_x, missrate=0.2):
        data = data_x.copy()
        rows, cols = data_x.shape
        total_no = rows*cols
        total_miss_no = np.round(total_no*missrate).astype(int)
        total_idx = self.idxarr.shape[0]
        idxarr = self.idxarr
        missarr = self.missarr
        #print(total_miss_no)
        miss_no = 0
        cum_no = self.idxarr[:,3:4]
        cum_no = cum_no.reshape((total_idx))
        cum_sum = np.max(cum_no)
        #print(cum_no)
        #print(total_idx)
        while True:
            loc_count = np.around(np.random.random()*cum_sum)
            #print('loc_count =', loc_count)
            idx = len(cum_no[cum_no <= loc_count])-1
            #print(cum_no[cum_no <= loc_count])
            #print('idx =', idx)
            startnan = idxarr[idx][0]
            nanlen = idxarr[idx][2]
            loc = np.around(np.random.random()*(rows-nanlen)).astype(int)
            #print('loc =', loc)
            #print(loc_count, idx)
            #print(idxarr[idx])
            #data_copy = data[loc:loc+nanlen].copy()
            data_copy = data[loc:loc+nanlen]
            #print('startnan=', startnan)
            #isnan = missarr[startnan:startnan+nanlen].copy()
            isnan = missarr[startnan:startnan+nanlen]
            #print('isnan =',isnan)
            miss_no += idxarr[idx][1]
            if (miss_no > total_miss_no):
                break
            data_copy[isnan==1] = np.nan
            data[loc:loc+nanlen] = data_copy
        #print('miss_data =', data)
        return data
    
    def save(data, max_tseq, save_dir='save'):
        no, dim = data.shape
        #print((no, dim))
        isnan = np.isnan(data).astype(int)
        isany = np.any(isnan, axis=1).astype(int)
        shifted = np.roll(isany, 1)
        shifted[0] = 1
        #print(isnan)
        #print(isany.astype(int))
        #print(shifted)
        startnan = ((isany == 1) & (shifted ==0)).astype(int)
        #print(startnan)
        group = startnan.cumsum()
        group = group*isany
#         print(group)
        n = np.max(group)
#         print(n)
        missarr = None
        cum_no = 0
        rowidx = 0
        for i in range(1, n+1):
            g = (group == i).astype(int)
            i = np.argmax(g)
            rows = g.sum()
            #print(len)
            #print(i)
            #print(type(missarr))
            if rows <= max_tseq:
                nanseq = isnan[i:i+rows, :]
                no = np.sum(nanseq)
                #print(no)
                if missarr is None:
                    missarr = nanseq
                    idxarr = np.array([[rowidx, no, rows, cum_no]])
                else:
                    missarr = np.concatenate((missarr, nanseq))
                    idxarr = np.concatenate((idxarr, [[rowidx, no, rows, cum_no]]), axis=0)
                cum_no += no
                rowidx += rows

        print(idxarr)
        miss_npy_file = os.path.join(save_dir, 'miss.npy')
        idx_npy_file = os.path.join(save_dir, 'idx.npy')
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)
        np.save(miss_npy_file, missarr)
        np.save(idx_npy_file, idxarr)
        print('miss_data file saved')

In [24]:
def interpolate(np_data, max_gap=3):
    #n = np_data.shape[1]
    data = pd.DataFrame(np_data)
    #data[0][0] = np.nan
    #data[0][1] = np.nan
    #data[0][2] = np.nan
    #data[data.columns[0]][0] = np.nan
    #data[data.columns[0]][1] = np.nan
    #data[data.columns[0]][2] = np.nan
    
    # create mask
    mask = data.copy()
    grp = ((mask.notnull() != mask.shift().notnull()).cumsum())
    grp['ones'] = 1
    for i in data.columns:
        mask[i] = (grp.groupby(i)['ones'].transform('count') < max_gap) | data[i].notnull()
    data = data.interpolate(method='polynomial', order=5, limit=max_gap, axis=0).bfill()[mask]
    return data.to_numpy()
    #return data
    
#filled_data = interpolate(norm_data, max_gap=3)
#np.arange(0, 5, dtype=int)
#['%d'%val for val in range(0,5)]

In [25]:
class GainDataGenerator(keras.utils.Sequence):
    'Generates data for GAIN'
    def __init__(self,
                 data_list,
                 batch_size=32,
                 input_width=24*3,
                 label_width=24*3,
                 shift=0,
                 fill_no=4,
                 miss_rate=0.2,
                 hint_rate=0.9,
                 normalize=True,
                 miss_pattern=None,
                 miss_data_load_dir='save',
                 alpha=100.):
        'Initialization'
        window_size = input_width
        
        # interpollation
        filled_data = []
        for data in data_list:
            data = interpolate(data, max_gap=fill_no)
            filled_data.append(data)
            
        data_list = filled_data
        
        # whole data
        self.data = np.concatenate(data_list)

        # TO-DO
        
        # pre calculation for  sequence data
        last_cum = 0
        cums = []
        for data in data_list:
            isnan = np.isnan(data)
            isany = np.any(isnan, axis=1)
            #shift same as pd.shift(isany, fill_value=True)
            shifted = np.roll(isany, 1)
            shifted[0] = True # set to nan
            
            start_seq = ((isany == False) & (shifted == True)).astype(int)
            cum = start_seq.cumsum()
            cum += last_cum
            last_cum = np.max(cum)
            cum[isany] = 0
            cums.append(cum)

        # Define mask matrix
        if miss_pattern is None:
            print("pattern none")
            self.data_m = binary_sampler(1-miss_rate, self.data.shape)
        else:
            #MissData.save(self.data, max_tseq = 12)
            print("load save")
            self.miss = MissData(load_dir=miss_data_load_dir)
            self.miss_rate = miss_rate
            miss_data = self.miss.make_missdata(self.data, self.miss_rate)
            self.data_m = 1. - np.isnan(miss_data).astype(float)
            
            self.data_m_rand = binary_sampler(1-(miss_rate/10.), self.data.shape)
            self.data_m[self.data_m_rand==0.] = 0.
        self.miss_pattern = miss_pattern
        
        # sequence data
        self.ids = np.concatenate(cums)
        data_idx = np.empty((0), dtype=int)
        for i in range(1, last_cum+1):
            seq_len = (self.ids == i).sum()
            start_id = np.argmax(self.ids == i)
            # possible data number in seqeunce
            time_len = seq_len - window_size + 1
            start_ids = np.arange(start_id, start_id+time_len)
            data_idx = np.append(data_idx, start_ids)
            
        # start index set for sequence data
        self.data_idx = data_idx
        self.input_width = input_width
        self.no = len(data_idx)
        
        #print('self.no = ', self.no)
        
        self.batch_size = batch_size
        
        # random shuffling  index
        self.batch_idx = sample_batch_index(self.no, self.no)
        self.batch_id = 0
        self.shape = (batch_size,self.input_width)+self.data.shape[1:]
        #self.hint_rate = hint_rate
            
    def __len__(self):
        'Denotes the number of batches per epoch'
        #return int(128/self.batch_size)
        #return 2
        return 1

    def __getitem__(self, index):
        'Generate one batch of data'
        #print('index =', index)
        # Sample batch
        x = np.empty((0, self.input_width, self.data.shape[1]))
        #m = np.empty((0, self.input_width, self.data.shape[1]))
        #h = np.empty((0, self.input_width, self.data.shape[1]))
        y = np.empty((0, self.input_width, self.data.shape[1]))
        #print(x.shape)
        #print(self.data.shape)
        #print(self.input_width)
        #self.batch_idx = sample_batch_index(self.no, self.batch_size)
        for cnt in range(0, self.batch_size):
            i = self.batch_idx[self.batch_id]
            self.batch_id += 1
            #self.batch_id %= self.batch_size
            self.batch_id %= self.no
            if self.miss_pattern and (self.batch_id == 0):
                self.batch_idx = sample_batch_index(self.no, self.no)
                miss_data = self.miss.make_missdata(self.data, self.miss_rate)
                self.data_m = 1. - np.isnan(miss_data).astype(float)
                self.data_m_rand = binary_sampler(1-self.miss_rate/10., self.data.shape)
                self.data_m[self.data_m_rand==0.] = 0.
            idx1 = self.data_idx[i]
            idx2 = self.data_idx[i]+self.input_width
            #print(idx1, idx2)
        
            Y_mb = self.data[idx1:idx2].copy()
            X_mb = Y_mb.copy()
            M_mb = self.data_m[idx1:idx2]
            Z_mb = uniform_sampler(0, 0.01, shape=X_mb.shape)
            X_mb = M_mb*X_mb + (1-M_mb)*Z_mb
            #H_mb_temp = binary_sampler(self.hint_rate, shape=X_mb.shape)
            #H_mb = M_mb * H_mb_temp
            X_mb[M_mb == 0] = np.nan
            Y_mb[M_mb == 1] = np.nan
            x = np.append(x, [X_mb], axis=0)
            #m = np.append(m, [M_mb], axis=0)
            #h = np.append(h, [H_mb], axis=0)
            y = np.append(y, [Y_mb], axis=0)
            
        #return [x, m, h], y
        return x, y
    
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        return

# dgen = GainDataGenerator(df)

In [26]:
class GAIN(keras.Model):
    def __init__(self, shape, alpha=100., load=False, hint_rate=0.9, gen_sigmoid=True, **kwargs):
        super(GAIN, self).__init__(**kwargs)
        self.shape = shape
        self.dim = np.prod(shape).astype(int)
        self.h_dim = self.dim
        self.gen_sigmoid = gen_sigmoid
        self.build_generator()
        self.build_discriminator()
        self.hint_rate = hint_rate
        self.alpha = alpha
        self.generator_optimizer = Adam()
        self.discriminator_optimizer = Adam()

    ## GAIN models
    def build_generator(self):
        last_activation = 'sigmoid' if self.gen_sigmoid else None
        xavier_initializer = tf.keras.initializers.GlorotNormal()

        shape = self.shape
        #x = Input(shape=(self.dim,), name='generator_input_x')
        #m = Input(shape=(self.dim,), name='generator_input_m')
        x = Input(shape=shape, name='generator_input_x')
        m = Input(shape=shape, name='generator_input_m')
        
        x_f = Flatten()(x)
        m_f = Flatten()(m)

        a = Concatenate()([x_f, m_f])

        a = Dense(self.h_dim, activation='relu', kernel_initializer=xavier_initializer)(a)
        #a = keras.layers.BatchNormalization()(a)
        a = Dense(self.h_dim, activation='relu', kernel_initializer=xavier_initializer)(a)
        #a = keras.layers.BatchNormalization()(a)
        a = Dense(self.dim, activation=last_activation, kernel_initializer=xavier_initializer)(a)
        G_prob = keras.layers.Reshape(shape)(a)
        self.generator = keras.models.Model([x, m], G_prob, name='generator')

    def build_discriminator(self):
        xavier_initializer = tf.keras.initializers.GlorotNormal()
        shape = self.shape

        #x = Input(shape=(self.dim,), name='discriminator_input_x')
        #h = Input(shape=(self.dim,), name='discriminator_input_h')
        x = Input(shape=shape, name='discriminator_input_x')
        h = Input(shape=shape, name='discriminator_input_h')
        
        x_f = Flatten()(x)
        h_f = Flatten()(h)

        a = Concatenate()([x_f, h_f])

        a = Dense(self.h_dim, activation='relu', kernel_initializer=xavier_initializer)(a)
        a = Dense(self.h_dim, activation='relu', kernel_initializer=xavier_initializer)(a)
        a = Dense(self.dim, activation='sigmoid', kernel_initializer=xavier_initializer)(a)
        D_prob = keras.layers.Reshape(shape)(a)
        self.discriminator = keras.models.Model([x, h], D_prob, name='discriminator')
        
    def call(self, inputs):
        if isinstance(inputs, tuple):
            inputs = inputs[0]
        shape = inputs.shape
        dims = np.prod(shape[1:])
        input_width = shape[1]
        #print('inputs.shape=',inputs.shape)
        x = inputs
        #x = x.reshape((n, -1))
        #print('dims=',dims)
        #x = keras.layers.Reshape((dims,))(x)
        #x = keras.layers.Reshape(tf.TensorShape((self.dim,)))(x)
        #print('x =', x)
        #print('x.shape = ', x.shape)
        #x = keras.layers.Reshape(tf.TensorShape([57]))(x)
        
        isnan = tf.math.is_nan(x)
        #m = 1.- keras.backend.cast(isnan, dtype=tf.float32)
        m = tf.where(isnan, 0., 1.)
        z = keras.backend.random_uniform(shape=tf.shape(x), minval=0.0, maxval=0.01)
        x = tf.where(isnan, z, x)
        #z = uniform_sampler(0, 0.01, shape=x.shape)
        #z = tf.keras.backend.random_uniform(shape=x.shape, minval=0.0, maxval=0.01)
        imputed_data = self.generator([x, m], training=False)
        #imputed_data = m*x + (1-m)*imputed_data
        #imputed_data = tf.where(isnan, imputed_data, np.nan)
        imputed_data = tf.where(isnan, imputed_data, x)
        #imputed_data = keras.layers.Reshape(shape[1:])(imputed_data)
        #print('imputed_data.shape = ', imputed_data.shape)
        
        return imputed_data
    
    def D_loss(M, D_prob):
        ## GAIN loss
        return -tf.reduce_mean(M * tf.keras.backend.log(D_prob + 1e-8) \
                         + (1-M) * tf.keras.backend.log(1. - D_prob + 1e-8))
    
    def G_loss(self, M, D_prob, X, G_sample):
        G_loss_temp = -tf.reduce_mean((1-M) * tf.keras.backend.log(D_prob + 1e-8))
        MSE_loss = tf.reduce_mean((M * X - M * G_sample)**2) / (tf.reduce_mean(M) + 1e-8)
        #G_loss_temp = GAIN.G_loss_bincross(M, D_prob)
        #MSE_loss = GAIN.MSE_loss(M, X, G_sample)
        G_loss = G_loss_temp + self.alpha * MSE_loss
        return G_loss
        
    def RMSE_loss(y_true, y_pred):
        isnan = tf.math.is_nan(y_true)
        M = tf.where(isnan, 1., 0.)
        return tf.sqrt(tf.reduce_sum(tf.where(isnan, 0., y_pred-y_true)**2)/tf.reduce_sum(1-M))
    
    def train_step(self, data):
        #[x, m, h], y = data
        x, y = data
        #X = keras.layers.Reshape((self.dim,), input_shape=self.shape)(x)
        #Y = keras.layers.Reshape((self.dim,), input_shape=self.shape)(y)
        #X = keras.layers.Flatten()(x)
        #Y = keras.layers.Flatten()(y)
        X = x
        Y = y
        #X = tf.reshape(x, shape=(x.shape[0], -1))
        #Y = tf.reshape(y, shape=(x.shape[0], -1))
        isnan = tf.math.is_nan(X)
        #M = 1 - keras.backend.cast(isnan, dtype=tf.float32)
        M = tf.where(isnan, 0., 1.)
        Z = keras.backend.random_uniform(shape=tf.shape(X), minval=0.0, maxval=0.01)
        #H_temp = binary_sampler(self.hint_rate, shape=X.shape)
        H_rand = keras.backend.random_uniform(shape=tf.shape(X), minval=0.0, maxval=1.)
        #H_temp = 1*keras.backend.cast((H_rand < self.hint_rate), dtype=tf.float32)
        H_temp = tf.where(H_rand < self.hint_rate, 1., 0.)
        
        H = M * H_temp
        #X = M * X + (1-M) * Z
        X = tf.where(isnan, Z, X)
        with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
            G_sample = self.generator([X, M], training=True)

            # Combine with observed data
            #Hat_X = tf.where(isnan, G_sample, X)
            Hat_X = X * M + G_sample * (1-M)
            D_prob = self.discriminator([Hat_X, H], training=True)
            gen_loss = self.G_loss(M, D_prob, X, G_sample)
            disc_loss = tf.keras.backend.mean(tf.keras.losses.binary_crossentropy(M, D_prob))
            #disc_loss = GAIN.D_loss(M, D_prob)
            #disc_loss = GAIN.D_loss(M, D_prob)

        gradients_of_generator = gen_tape.gradient(gen_loss, self.generator.trainable_variables)
        gradients_of_discriminator = disc_tape.gradient(disc_loss, self.discriminator.trainable_variables)

        self.generator_optimizer.apply_gradients(zip(gradients_of_generator, self.generator.trainable_variables))
        self.discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, self.discriminator.trainable_variables))
        
        rmse = tf.sqrt(tf.reduce_sum(tf.where(isnan, G_sample - Y, 0.)**2)/tf.reduce_sum(1-M))
        return {
                 'gen_loss':     gen_loss,
                 'disc_loss':    disc_loss,
                 'rmse':         rmse,
               }
    
    def save(self, save_dir='save'):
        if not os.path.exists(save_dir):
          os.makedirs(save_dir)
        disc_savefile = os.path.join(save_dir, 'discriminator.h5')
        gen_savefile = os.path.join(save_dir, 'generator.h5')
        self.discriminator.save_weights(disc_savefile)
        self.generator.save_weights(gen_savefile)

    def load(self, save_dir='save'):
        disc_savefile = os.path.join(save_dir, 'discriminator.h5')
        gen_savefile = os.path.join(save_dir, 'generator.h5')
        try:
          self.discriminator.load_weights(disc_savefile)
          self.generator.load_weights(gen_savefile)
          print('model weights loaded')
        except:
          print('model loadinng error')

In [27]:
def make_dataset_gain(self, data):
  dg = GainDataGenerator(
      df,
      input_width = self.input_width,
      label_width = self.label_width,
      batch_size = 128,
      normalize = False,
      miss_pattern = True,
      miss_rate = 0.15,
      fill_no = 3,
  )
  self.dg = dg
  ds = tf.data.Dataset.from_generator(
      lambda: dg,
      output_types=(tf.float32, tf.float32),
      output_shapes=(
        dg.shape,
        dg.shape
        #[batch_size, train_generator.dim],
        #[batch_size, train_generator.dim],
      )
  )
  return ds

WindowGenerator.make_dataset = make_dataset_gain

# 측정망별 반복 구간 

In [126]:
df

[             수온    수소이온농도     전기전도도      용존산소     총유기탄소       총질소        총인  \
 0     -1.189365 -0.136485  0.622171  0.736093 -0.476043  0.712601 -0.233359   
 1     -1.200971 -0.136485  0.633353  0.736093 -0.302378  0.657585 -0.233359   
 2     -1.200971 -0.136485  0.633353  0.708933 -0.302378  0.597516 -0.254338   
 3     -1.212576 -0.136485  0.633353  0.681773 -0.302378  0.655901 -0.254338   
 4     -1.212576 -0.136485  0.622171  0.654613 -0.389211  0.722145 -0.254338   
 ...         ...       ...       ...       ...       ...       ...       ...   
 35059       NaN       NaN       NaN       NaN       NaN       NaN       NaN   
 35060       NaN       NaN       NaN       NaN       NaN       NaN       NaN   
 35061       NaN       NaN       NaN       NaN       NaN       NaN       NaN   
 35062       NaN       NaN       NaN       NaN       NaN       NaN       NaN   
 35063       NaN       NaN       NaN       NaN       NaN       NaN       NaN   
 
          클로로필-a       Day sin   Day c

In [28]:
norm_df = pd.concat(df,axis=0)
norm_data = norm_df.to_numpy()
MissData.save(norm_data, max_tseq = 24)
# 1: 자동, 2: 수질, 

[[    0     9     2     0]
 [    2     5     5     9]
 [    7    22     4    14]
 ...
 [ 9914     5     1 32856]
 [ 9915     5     1 32861]
 [ 9916    15     3 32866]]
miss_data file saved


In [127]:
norm_df

,수온,수소이온농도,전기전도도,용존산소,총유기탄소,총질소,총인,클로로필-a,Day sin,Day cos,Year sin,Year cos
0,-1.189365,-0.136485,0.622171,0.736093,-0.476043,0.712601,-0.233359,0.915127,-1.999588e-12,1.414209,-0.003771,1.414160
1,-1.200971,-0.136485,0.633353,0.736093,-0.302378,0.657585,-0.233359,0.899607,3.660241e-01,1.366021,-0.002757,1.414162
2,-1.200971,-0.136485,0.633353,0.708933,-0.302378,0.597516,-0.254338,0.860809,7.071043e-01,1.224741,-0.001743,1.414164
3,-1.212576,-0.136485,0.633353,0.681773,-0.302378,0.655901,-0.254338,0.832356,9.999964e-01,0.999996,-0.000730,1.414165
4,-1.212576,-0.136485,0.622171,0.654613,-0.389211,0.722145,-0.254338,0.796144,1.224741e+00,0.707104,0.000284,1.414165
...,...,...,...,...,...,...,...,...,...,...,...,...
35059,-1.409869,-0.267080,0.666901,1.333618,-0.562875,-0.782943,-0.527066,-0.851513,-1.366021e+00,0.366024,-0.008109,1.414142
35060,-1.421475,-0.267080,0.633353,1.387939,-0.476043,-0.711085,-0.527066,-0.848926,-1.224741e+00,0.707104,-0.007096,1.414147
35061,-1.433080,-0.136485,0.610988,1.469419,-0.476043,-0.708278,-0.548046,-0.851513,-9.999964e-01,0.999996,-0.006082,1.414152
35062,-1.444686,-0.136485,0.599806,1.469419,-0.476043,-0.831784,-0.548046,-0.851513,-7.071043e-01,1.224741,-0.005068,1.414156


In [29]:
dgen = GainDataGenerator(df)

pattern none


In [30]:
train_df = df_all
val_df = df_all
test_df = df_all
df_all

,수온,수소이온농도,전기전도도,용존산소,총유기탄소,총질소,총인,클로로필-a,Day sin,Day cos,Year sin,Year cos
0,7.3,7.3,346.0,11.5,3.3,4.826,0.080,72.3,-1.407890e-12,1.000000,-0.002666,0.999996
1,7.2,7.3,347.0,11.5,3.5,4.728,0.080,71.7,2.588190e-01,0.965926,-0.001950,0.999998
2,7.2,7.3,347.0,11.4,3.5,4.621,0.079,70.2,5.000000e-01,0.866025,-0.001233,0.999999
3,7.1,7.3,347.0,11.3,3.5,4.725,0.079,69.1,7.071068e-01,0.707107,-0.000516,1.000000
4,7.1,7.3,346.0,11.2,3.4,4.843,0.079,67.7,8.660254e-01,0.500000,0.000201,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
35059,5.4,7.2,350.0,13.7,3.2,2.162,0.066,4.0,-9.659258e-01,0.258819,-0.005734,0.999984
35060,5.3,7.2,347.0,13.9,3.3,2.290,0.066,4.1,-8.660254e-01,0.500000,-0.005017,0.999987
35061,5.2,7.3,345.0,14.2,3.3,2.295,0.065,4.0,-7.071068e-01,0.707107,-0.004301,0.999991
35062,5.1,7.3,344.0,14.2,3.3,2.075,0.065,4.0,-5.000000e-01,0.866025,-0.003584,0.999994


In [31]:
wide_window = WindowGenerator(
    input_width=24*5, label_width=24*5, shift=0,
    #label_columns=['T (degC)']
)

wide_window

Total window size: 120
Input indices: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119]
Label indices: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 

## Graph 

In [32]:
wide_window.example[0].shape

load save


TensorShape([128, 120, 12])

In [33]:
wide_window.plot(plot_col=target_col)
print('make_dataset_gain: dg.no = ', wide_window.dg.no)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

make_dataset_gain: dg.no =  11525


In [34]:
plt.figure(figsize=(9,10))
isnan = np.isnan(norm_data).astype(int)
data = isnan
n = data.shape[0]
seq_len = n//8
for i in range(8):
    plt.subplot(181+i)
    plt.imshow(data[i*seq_len:(i+1)*seq_len, 0:], aspect='auto')
    plt.yticks([])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## 컴파일 및 학습_GAIN

In [35]:
val_performance = {}
performance = {}

In [36]:
gain = GAIN(shape=wide_window.dg.shape[1:], gen_sigmoid=False)
gain.compile(loss=GAIN.RMSE_loss)

In [37]:
MAX_EPOCHS = 2000

def compile_and_fit(model, window, patience=10):
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    mode='min')

  #model.compile(loss=tf.losses.MeanSquaredError(),
                #optimizer=tf.optimizers.Adam(),
                #metrics=[tf.metrics.MeanAbsoluteError()])
  model.compile(loss=GAIN.RMSE_loss)

  history = model.fit(window.train, epochs=MAX_EPOCHS,
                      validation_data=window.val,
                      callbacks=[early_stopping])
  return history

**모델 학습**

In [38]:
history = compile_and_fit(gain, wide_window, patience=MAX_EPOCHS//5)


val_performance['Gain'] = gain.evaluate(wide_window.val)
performance['Gain'] = gain.evaluate(wide_window.test, verbose=0)

# gain.save(save_dir='save')

load save
load save
Epoch 1/2000
1/1 [==============================] - 1s 1s/step - gen_loss: 122.2996 - disc_loss: 0.7258 - rmse: 1.0595 - val_loss: 0.9444
Epoch 2/2000
1/1 [==============================] - 0s 138ms/step - gen_loss: 98.4579 - disc_loss: 0.5455 - rmse: 0.9951 - val_loss: 0.9058
Epoch 3/2000
1/1 [==============================] - 0s 110ms/step - gen_loss: 85.2968 - disc_loss: 0.4894 - rmse: 0.9201 - val_loss: 0.8396
Epoch 4/2000
1/1 [==============================] - 0s 125ms/step - gen_loss: 72.5988 - disc_loss: 0.4133 - rmse: 0.8493 - val_loss: 0.8011
Epoch 5/2000
1/1 [==============================] - 0s 137ms/step - gen_loss: 65.8723 - disc_loss: 0.3896 - rmse: 0.7882 - val_loss: 0.7528
Epoch 6/2000
1/1 [==============================] - 0s 143ms/step - gen_loss: 54.6895 - disc_loss: 0.4058 - rmse: 0.7391 - val_loss: 0.7020
Epoch 7/2000
1/1 [==============================] - 0s 136ms/step - gen_loss: 48.0374 - disc_loss: 0.3908 - rmse: 0.7076 - val_loss: 0.6466
Ep

1/1 [==============================] - 0s 142ms/step - gen_loss: 6.6000 - disc_loss: 0.2752 - rmse: 0.3291 - val_loss: 0.3057
Epoch 60/2000
1/1 [==============================] - 0s 114ms/step - gen_loss: 5.8375 - disc_loss: 0.2650 - rmse: 0.2859 - val_loss: 0.3561
Epoch 61/2000
1/1 [==============================] - 0s 135ms/step - gen_loss: 6.2316 - disc_loss: 0.2596 - rmse: 0.3447 - val_loss: 0.3333
Epoch 62/2000
1/1 [==============================] - 0s 106ms/step - gen_loss: 6.1794 - disc_loss: 0.2661 - rmse: 0.3129 - val_loss: 0.3316
Epoch 63/2000
1/1 [==============================] - 0s 152ms/step - gen_loss: 6.4832 - disc_loss: 0.2724 - rmse: 0.3436 - val_loss: 0.3215
Epoch 64/2000
1/1 [==============================] - 0s 131ms/step - gen_loss: 6.0875 - disc_loss: 0.2660 - rmse: 0.3350 - val_loss: 0.3220
Epoch 65/2000
1/1 [==============================] - 0s 113ms/step - gen_loss: 5.3252 - disc_loss: 0.2610 - rmse: 0.2857 - val_loss: 0.3153
Epoch 66/2000
1/1 [===============

1/1 [==============================] - 0s 121ms/step - gen_loss: 4.8381 - disc_loss: 0.2217 - rmse: 0.2884 - val_loss: 0.3262
Epoch 118/2000
1/1 [==============================] - 0s 105ms/step - gen_loss: 4.8415 - disc_loss: 0.2196 - rmse: 0.2894 - val_loss: 0.3106
Epoch 119/2000
1/1 [==============================] - 0s 131ms/step - gen_loss: 4.6310 - disc_loss: 0.2280 - rmse: 0.2876 - val_loss: 0.2850
Epoch 120/2000
1/1 [==============================] - 0s 172ms/step - gen_loss: 4.6952 - disc_loss: 0.2235 - rmse: 0.2876 - val_loss: 0.3242
Epoch 121/2000
1/1 [==============================] - 0s 144ms/step - gen_loss: 4.4974 - disc_loss: 0.2199 - rmse: 0.2856 - val_loss: 0.3240
Epoch 122/2000
1/1 [==============================] - 0s 137ms/step - gen_loss: 4.5651 - disc_loss: 0.2224 - rmse: 0.3026 - val_loss: 0.3147
Epoch 123/2000
1/1 [==============================] - 0s 127ms/step - gen_loss: 4.6635 - disc_loss: 0.2137 - rmse: 0.2766 - val_loss: 0.2980
Epoch 124/2000
1/1 [========

1/1 [==============================] - 0s 130ms/step - gen_loss: 3.7509 - disc_loss: 0.2034 - rmse: 0.2641 - val_loss: 0.2987
Epoch 176/2000
1/1 [==============================] - 0s 98ms/step - gen_loss: 3.5888 - disc_loss: 0.2057 - rmse: 0.2522 - val_loss: 0.3078
Epoch 177/2000
1/1 [==============================] - 0s 133ms/step - gen_loss: 3.4110 - disc_loss: 0.2045 - rmse: 0.2422 - val_loss: 0.2891
Epoch 178/2000
1/1 [==============================] - 0s 117ms/step - gen_loss: 3.6502 - disc_loss: 0.1986 - rmse: 0.2537 - val_loss: 0.3548
Epoch 179/2000
1/1 [==============================] - 0s 117ms/step - gen_loss: 3.5689 - disc_loss: 0.2074 - rmse: 0.2462 - val_loss: 0.2791
Epoch 180/2000
1/1 [==============================] - 0s 121ms/step - gen_loss: 4.0239 - disc_loss: 0.1993 - rmse: 0.2777 - val_loss: 0.3032
Epoch 181/2000
1/1 [==============================] - 1s 505ms/step - gen_loss: 4.4463 - disc_loss: 0.2073 - rmse: 0.2825 - val_loss: 0.2720
Epoch 182/2000
1/1 [=========

1/1 [==============================] - 0s 149ms/step - gen_loss: 3.1569 - disc_loss: 0.1864 - rmse: 0.2588 - val_loss: 0.2665
Epoch 234/2000
1/1 [==============================] - 0s 118ms/step - gen_loss: 3.4763 - disc_loss: 0.1897 - rmse: 0.2524 - val_loss: 0.2649
Epoch 235/2000
1/1 [==============================] - 0s 120ms/step - gen_loss: 3.4068 - disc_loss: 0.1926 - rmse: 0.2541 - val_loss: 0.2519
Epoch 236/2000
1/1 [==============================] - 0s 139ms/step - gen_loss: 3.3748 - disc_loss: 0.1919 - rmse: 0.2402 - val_loss: 0.2644
Epoch 237/2000
1/1 [==============================] - 0s 100ms/step - gen_loss: 3.4333 - disc_loss: 0.1870 - rmse: 0.2495 - val_loss: 0.2674
Epoch 238/2000
1/1 [==============================] - 0s 124ms/step - gen_loss: 3.3086 - disc_loss: 0.1834 - rmse: 0.2550 - val_loss: 0.2518
Epoch 239/2000
1/1 [==============================] - 0s 146ms/step - gen_loss: 3.4148 - disc_loss: 0.1847 - rmse: 0.2555 - val_loss: 0.2564
Epoch 240/2000
1/1 [========

...

1/1 [==============================] - 0s 123ms/step - gen_loss: 1.9457 - disc_loss: 0.1088 - rmse: 0.2090 - val_loss: 0.2253
Epoch 1733/2000
1/1 [==============================] - 0s 140ms/step - gen_loss: 2.2035 - disc_loss: 0.1132 - rmse: 0.2177 - val_loss: 0.2364
Epoch 1734/2000
1/1 [==============================] - 0s 152ms/step - gen_loss: 2.2169 - disc_loss: 0.1129 - rmse: 0.2168 - val_loss: 0.2218
Epoch 1735/2000
1/1 [==============================] - 0s 135ms/step - gen_loss: 2.3160 - disc_loss: 0.1161 - rmse: 0.2313 - val_loss: 0.2326
Epoch 1736/2000
1/1 [==============================] - 0s 132ms/step - gen_loss: 2.0529 - disc_loss: 0.1107 - rmse: 0.2179 - val_loss: 0.2376
Epoch 1737/2000
1/1 [==============================] - 0s 145ms/step - gen_loss: 2.1408 - disc_loss: 0.1132 - rmse: 0.2030 - val_loss: 0.2455
Epoch 1738/2000
1/1 [==============================] - 0s 144ms/step - gen_loss: 2.2474 - disc_loss: 0.1115 - rmse: 0.2212 - val_loss: 0.2759
Epoch 1739/2000
1/1 [=

1/1 [==============================] - 0s 143ms/step - gen_loss: 1.9261 - disc_loss: 0.1122 - rmse: 0.2055 - val_loss: 0.2254
Epoch 1790/2000
1/1 [==============================] - 0s 129ms/step - gen_loss: 1.7872 - disc_loss: 0.1105 - rmse: 0.1772 - val_loss: 0.2077
Epoch 1791/2000
1/1 [==============================] - 0s 129ms/step - gen_loss: 1.7669 - disc_loss: 0.1128 - rmse: 0.1809 - val_loss: 0.2544
Epoch 1792/2000
1/1 [==============================] - 0s 148ms/step - gen_loss: 1.6890 - disc_loss: 0.1059 - rmse: 0.1928 - val_loss: 0.2119
Epoch 1793/2000
1/1 [==============================] - 0s 147ms/step - gen_loss: 1.7208 - disc_loss: 0.1120 - rmse: 0.1932 - val_loss: 0.2456
Epoch 1794/2000
1/1 [==============================] - 0s 126ms/step - gen_loss: 1.7946 - disc_loss: 0.1145 - rmse: 0.2162 - val_loss: 0.2213
Epoch 1795/2000
1/1 [==============================] - 0s 129ms/step - gen_loss: 1.7574 - disc_loss: 0.1133 - rmse: 0.1990 - val_loss: 0.2115
Epoch 1796/2000
1/1 [=

1/1 [==============================] - 0s 160ms/step - gen_loss: 1.8560 - disc_loss: 0.1092 - rmse: 0.2263 - val_loss: 0.2164
Epoch 1847/2000
1/1 [==============================] - 0s 132ms/step - gen_loss: 1.8425 - disc_loss: 0.1131 - rmse: 0.2203 - val_loss: 0.2025
Epoch 1848/2000
1/1 [==============================] - 0s 128ms/step - gen_loss: 1.8332 - disc_loss: 0.1105 - rmse: 0.2038 - val_loss: 0.2128
Epoch 1849/2000
1/1 [==============================] - 0s 93ms/step - gen_loss: 1.8372 - disc_loss: 0.1105 - rmse: 0.2147 - val_loss: 0.2078
Epoch 1850/2000
1/1 [==============================] - 0s 156ms/step - gen_loss: 1.9209 - disc_loss: 0.1083 - rmse: 0.1999 - val_loss: 0.1977
Epoch 1851/2000
1/1 [==============================] - 0s 136ms/step - gen_loss: 1.8656 - disc_loss: 0.1117 - rmse: 0.2113 - val_loss: 0.2172
Epoch 1852/2000
1/1 [==============================] - 0s 119ms/step - gen_loss: 1.7404 - disc_loss: 0.1073 - rmse: 0.2214 - val_loss: 0.2092
Epoch 1853/2000
1/1 [==

Epoch 1904/2000
1/1 [==============================] - 0s 149ms/step - gen_loss: 1.9141 - disc_loss: 0.1128 - rmse: 0.2051 - val_loss: 0.2100
Epoch 1905/2000
1/1 [==============================] - 0s 117ms/step - gen_loss: 1.8073 - disc_loss: 0.1150 - rmse: 0.1750 - val_loss: 0.2044
Epoch 1906/2000
1/1 [==============================] - 0s 149ms/step - gen_loss: 1.8269 - disc_loss: 0.1080 - rmse: 0.2027 - val_loss: 0.2178
Epoch 1907/2000
1/1 [==============================] - 0s 133ms/step - gen_loss: 2.0501 - disc_loss: 0.1146 - rmse: 0.2414 - val_loss: 0.2070
Epoch 1908/2000
1/1 [==============================] - 0s 132ms/step - gen_loss: 1.9547 - disc_loss: 0.1108 - rmse: 0.2019 - val_loss: 0.2263
Epoch 1909/2000
1/1 [==============================] - 0s 115ms/step - gen_loss: 1.8421 - disc_loss: 0.1132 - rmse: 0.2078 - val_loss: 0.2056
Epoch 1910/2000
1/1 [==============================] - 0s 117ms/step - gen_loss: 1.9316 - disc_loss: 0.1096 - rmse: 0.2054 - val_loss: 0.2122
Epoch 

1/1 [==============================] - 0s 152ms/step - gen_loss: 1.6548 - disc_loss: 0.1085 - rmse: 0.1947 - val_loss: 0.2187
Epoch 1962/2000
1/1 [==============================] - 0s 129ms/step - gen_loss: 1.6800 - disc_loss: 0.1070 - rmse: 0.1801 - val_loss: 0.1981
Epoch 1963/2000
1/1 [==============================] - 0s 123ms/step - gen_loss: 1.6450 - disc_loss: 0.1108 - rmse: 0.1985 - val_loss: 0.2357
Epoch 1964/2000
1/1 [==============================] - 0s 142ms/step - gen_loss: 1.7069 - disc_loss: 0.1065 - rmse: 0.1823 - val_loss: 0.1928
Epoch 1965/2000
1/1 [==============================] - 0s 126ms/step - gen_loss: 1.6488 - disc_loss: 0.1083 - rmse: 0.1824 - val_loss: 0.2116
Epoch 1966/2000
1/1 [==============================] - 0s 101ms/step - gen_loss: 1.6715 - disc_loss: 0.1108 - rmse: 0.1890 - val_loss: 0.2261
Epoch 1967/2000
1/1 [==============================] - 0s 130ms/step - gen_loss: 1.6174 - disc_loss: 0.1098 - rmse: 0.2131 - val_loss: 0.2120
Epoch 1968/2000
1/1 [=

**학습 loss history 출력**

In [39]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax2 = ax.twinx()
ax.plot(history.history['gen_loss'], label='gen_loss')
ax.plot(history.history['disc_loss'], label='disc_loss')
ax2.plot(history.history['rmse'], label='rmse', color='green')
ax2.plot(history.history['val_loss'], label='val_loss', color='red')
#plt.legend(history.history.keys(), loc='upper right')
#ax.legend(loc='upper center')
ax.legend(loc='upper center')
ax2.legend(loc='upper right')
ax.set_xlabel("epochs")
ax.set_ylabel("loss")
ax2.set_ylabel("rmse")
plt.show()    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**성능 측정**

In [40]:
gain.evaluate(wide_window.test.repeat(), steps=100)

load save
100/100 [==============================] - 3s 30ms/step - loss: 0.2258


0.22581180930137634

**model save**

In [41]:
gain.save(save_dir='save')

**샘플 prediction 출력**

In [42]:
wide_window.plot(gain, plot_col='총유기탄소')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [43]:
norm_df = pd.concat(df,axis=0)

In [44]:
data = norm_df.to_numpy()

total_n = wide_window.dg.data.shape[0]
print(total_n)
unit_shape = wide_window.dg.shape[1:]
print(unit_shape)
#dim = np.prod(wide_window.dg.shape[1:]).astype(int)
dim = wide_window.dg.shape[1]
print(dim)
n = (total_n//dim)*dim

x = data[0:n].copy()
y_true = data[0:n].copy()

#x = interpolate(x, max_gap=3)

print('x.shape =', x.shape)
x_reshape = x.reshape((-1,)+unit_shape)
print('x_reshape.shape =', x_reshape.shape)
isnan = np.isnan(x_reshape)
print(isnan.sum())
print('y_true.shape=', y_true.shape)
isnan = np.isnan(y_true)
print(isnan.sum())

x_remain = data[-wide_window.dg.shape[1]:].copy()
x_remain_reshape = x_remain.reshape((-1,)+unit_shape)
x_remain_reshape.shape

140256
(120, 12)
120
x.shape = (140160, 12)
x_reshape.shape = (1168, 120, 12)
240597
y_true.shape= (140160, 12)
240597


(1, 120, 12)

In [45]:
gain.evaluate(x_reshape, y_true.reshape((-1,)+unit_shape))

37/37 [==============================] - 0s 3ms/step - loss: 0.0000e+00


0.0

In [46]:
y_pred = gain.predict(x_reshape)
y_remain_pred = gain.predict(x_remain_reshape)

In [47]:
y_pred = y_pred.reshape(y_true.shape)
y_remain_pred = y_remain_pred.reshape(x_remain.shape)
print(y_pred.shape, y_remain_pred.shape)
y_pred = np.append(y_pred, y_remain_pred[-(total_n-n):], axis=0)
print(y_pred.shape)

(140160, 12) (120, 12)
(140256, 12)


In [48]:
y_pred[~np.isnan(data)] = np.nan

In [49]:
n = 8
plt.figure(figsize=(9,20))
for i in range(n):
    #plt.subplot('%d1%d'%(n,i))
    plt.subplot(811+i)
    plt.plot(x[:, i])
    plt.plot(y_pred[:, i])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**Imputed Data**

In [50]:
total_n = wide_window.dg.data.shape[0]
print(total_n)
unit_shape = wide_window.dg.shape[1:]
print('unit_shape=', unit_shape)
time_seq = unit_shape[0]
print(time_seq)
n = (total_n//time_seq)*time_seq
print('n=', n)

gans = []
oris = []
for i in range(len(df)):
    x = df[i].to_numpy()
    total_n = x.shape[0]
    n = (total_n//time_seq)*time_seq
    x = x[0:n]
    x_block = x.reshape((-1,)+unit_shape)
    y = gain.predict(x_block)
    y_gan = y.reshape(x.shape)
    
    # cut off sin, cos data
    if (i > 0):
        x = x[:, :-4]
        y_gan = y_gan[:, :-4]
    gans.append(y_gan)
    oris.append(x)
print(x.shape)
print(y_gan.shape)

140256
unit_shape= (120, 12)
120
n= 140160
(35040, 8)
(35040, 8)


# 예측

In [51]:
df[0].head

<bound method NDFrame.head of              수온    수소이온농도     전기전도도      용존산소     총유기탄소       총질소        총인  \
0     -1.189365 -0.136485  0.622171  0.736093 -0.476043  0.712601 -0.233359   
1     -1.200971 -0.136485  0.633353  0.736093 -0.302378  0.657585 -0.233359   
2     -1.200971 -0.136485  0.633353  0.708933 -0.302378  0.597516 -0.254338   
3     -1.212576 -0.136485  0.633353  0.681773 -0.302378  0.655901 -0.254338   
4     -1.212576 -0.136485  0.622171  0.654613 -0.389211  0.722145 -0.254338   
...         ...       ...       ...       ...       ...       ...       ...   
35059       NaN       NaN       NaN       NaN       NaN       NaN       NaN   
35060       NaN       NaN       NaN       NaN       NaN       NaN       NaN   
35061       NaN       NaN       NaN       NaN       NaN       NaN       NaN   
35062       NaN       NaN       NaN       NaN       NaN       NaN       NaN   
35063       NaN       NaN       NaN       NaN       NaN       NaN       NaN   

         클로로필-a      

In [52]:
ori = np.concatenate(oris, axis=1)
gan = np.concatenate(gans, axis=1)
# print(oris[0].shape, gans[0].shape)
# print(oris[1].shape, gans[1].shape)
# print(ori.shape, gan.shape)

total_no = ori.shape[0]
train_no = int(total_no*0.8)
val_no = int(total_no * 0.9)

train_slice = slice(0, train_no)
val_slice = slice(train_no, val_no)
test_slice = slice(val_no, None)

train_df = pd.DataFrame(gan[train_slice])
val_df = pd.DataFrame(gan[val_slice])
test_df = pd.DataFrame(gan[test_slice])

train_ori_df = pd.DataFrame(ori[train_slice])
val_ori_df = pd.DataFrame(ori[val_slice])
test_ori_df = pd.DataFrame(ori[test_slice])

num_features = train_df.shape[1]
#out_num_features = oris[0].shape[1]-4
# out_features = [target_col_idx]
# out_num_features = len(out_features)
# out_num_features

In [53]:
train_df 

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,-1.189365,-0.136485,0.622171,0.736093,-0.476043,0.712601,-0.233359,0.915127,-1.999588e-12,1.414209,...,-0.841753,-0.714423,-1.317026,-0.005889,0.163686,0.980535,-0.997037,-0.676840,-0.946648,-0.817887
1,-1.200971,-0.136485,0.633353,0.736093,-0.302378,0.657585,-0.233359,0.899607,3.660241e-01,1.366021,...,-0.820774,-0.727356,-1.340236,-0.136485,0.152504,0.871895,-0.910205,-0.681893,-1.009585,-0.820473
2,-1.200971,-0.136485,0.633353,0.708933,-0.302378,0.597516,-0.254338,0.860809,7.071043e-01,1.224741,...,-0.820774,-0.722183,-1.363447,-0.136485,0.152504,0.763254,-0.823372,-0.663367,-1.072523,-0.823060
3,-1.212576,-0.136485,0.633353,0.681773,-0.302378,0.655901,-0.254338,0.832356,9.999964e-01,0.999996,...,-0.883711,-0.719597,-1.386658,-0.136485,0.118956,0.736093,-0.823372,-0.655508,-1.114481,-0.820473
4,-1.212576,-0.136485,0.622171,0.654613,-0.389211,0.722145,-0.254338,0.796144,1.224741e+00,0.707104,...,-0.925669,-0.729943,-1.398264,-0.267080,0.118956,0.708933,-0.910205,-0.671788,-1.114481,-0.820473
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28027,-0.748357,-0.267080,1.259576,-0.133034,1.173772,2.211514,0.039369,-0.132441,-1.366021e+00,0.366024,...,2.081025,0.886675,-0.806385,1.561252,0.812274,1.442259,2.042096,-0.753751,0.689721,0.604736
28028,-0.748357,-0.267080,1.270758,-0.187354,1.086940,2.159304,0.039369,-0.145374,-1.224741e+00,0.707104,...,3.417002,0.902194,-0.794779,1.430657,0.957647,1.387939,2.042096,-0.750944,0.668742,0.591803
28029,-0.748357,-0.267080,1.270758,-0.214514,1.086940,2.099235,0.060348,-0.173827,-9.999964e-01,0.999996,...,2.766650,0.907367,-0.794779,1.430657,1.058290,1.333618,2.042096,-0.763295,0.668742,0.581457
28030,-0.759963,-0.267080,1.281941,-0.268835,1.173772,2.073411,0.039369,-0.225558,-7.071043e-01,1.224741,...,4.256166,0.871155,-0.794779,1.430657,0.991195,1.224977,1.868431,-0.769470,0.668742,0.594390


In [54]:
df[0].columns

Index(['수온', '수소이온농도', '전기전도도', '용존산소', '총유기탄소', '총질소', '총인', '클로로필-a',
       'Day sin', 'Day cos', 'Year sin', 'Year cos'],
      dtype='object')

In [55]:
MAX_EPOCHS = 150
target_col = '용존산소'
num_features = df[0].shape[1]
label_columns_indices = {name: i for i, name in enumerate(df[0])}
target_col_idx = label_columns_indices[target_col]
# target_col_idx
# out_features = [target_col_idx]
out_features = [3,4,5,6,7]
out_num_features = len(out_features)
out_num_features

5

In [56]:
class WaterDataGenerator(keras.utils.Sequence):
    'Generates data for water'
    def __init__(self,
                 imputed_data,
                 ori_data = None,
                 batch_size=32,
                 input_width=24*7,
                 label_width=24*3,
                 shift=24*3,
                 skip_time = None,
                 shuffle = True,
                 out_features = None,
                 out_num_features = None,
                ):
        'Initialization'
        self.window_size = input_width+shift
        self.total_no = imputed_data.shape[0]
        self.data = imputed_data
        self.input_width = input_width
        self.label_width = label_width
        self.batch_size = batch_size
        self.input_shape = (batch_size, input_width, self.data.shape[1])
        self.out_num_features = out_num_features
#         print("out_features")
#         print(out_features)
        if out_features:
            self.out_features = out_features
        else:
            self.out_features = [i for i in range(out_num_features)]
        self.label_shape = (batch_size, label_width, self.out_num_features)
        if (skip_time):
            # TO-DO
            self.no = self.total_no - self.window_size
            self.data_idx = np.arange(0, self.no)
        else:
            self.no = self.total_no - self.window_size
            self.data_idx = np.arange(0, self.no)
            
        if shuffle:
            self.batch_idx = np.random.permutation(self.no)
        else:
            self.batch_idx = np.arange(0, self.no)
        self.batch_id = 0
        
        
    def __len__(self):
        'Denotes the number of batches per epoch'
        #return int(128/self.batch_size)
        #return 2
        return 1

    def __getitem__(self, index):
        'Generate one batch of data'
        #print('index =', index)
        #print('self.no =', self.no)
        #print('self.total_no =', self.total_no)
        #print('self.batch_id =', self.batch_id)
        # Sample batch
        label_width = self.label_width
        batch_idx = self.batch_idx
        
        x = np.empty((0, self.input_width, self.data.shape[1]))
        
        #y = np.empty((0, self.input_width, self.data.shape[1]))
        y = np.empty((0, self.label_width, self.out_num_features))
       
        #print(x.shape)
        #print(y.shape)
        
        
        for cnt in range(0, self.batch_size):
            i = self.batch_id
            self.batch_id += 1
            idx1 = self.data_idx[batch_idx[i]]
            idx2 = idx1 + self.input_width
            
            X = self.data[idx1:idx2].to_numpy()
            
            
            idx1 = self.data_idx[batch_idx[i]] + self.window_size - label_width
            idx2 = idx1 + label_width
            
            #Y = self.data[idx1:idx2,:,:out_num_features]
            Y = self.data.iloc[idx1:idx2, self.out_features].to_numpy()
            #Y = self.data[idx1:idx2]
            #print('Y.shape = ', Y.shape)
            #Y = Y.iloc[:,:out_num_features]
            
            self.batch_id %= self.no
            #print("x.shape=", x.shape)
            #print('X.shape=', X.shape)
            #print(type(x), type(X))
            x = np.append(x, [X], axis = 0)
            
            y = np.append(y, [Y], axis = 0)
            
        return x, y
    
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        return

**WaterDataGenerator 테스트**

In [57]:
OUT_STEPS = 24*5

In [58]:
# OUT_STEPS = 24*3
# wdg = WaterDataGenerator(train_df,
#                          batch_size=128,
#                          input_width = 24*7,
#                          label_width = OUT_STEPS,
#                          shift = OUT_STEPS,
#                          out_num_features = out_num_features,
# #                          out_features = out_features
#                         )

In [59]:
# train_df

In [60]:
# it = iter(wdg)

In [61]:
# x,y = next(it)
# x.shape, y.shape

In [62]:
def make_dataset_water(self, data):
#   print("make_dataset_water")
#   print(out_features)
  dg = WaterDataGenerator(
      data,
      batch_size = 128,
      input_width = self.input_width,
      label_width = self.label_width,
      shift = self.label_width,
      out_features = out_features,
      out_num_features = out_num_features,
  )
  #self.dg = dg
  ds = tf.data.Dataset.from_generator(
      lambda: dg,
      output_types=(tf.float32, tf.float32),
      output_shapes=(
        dg.input_shape,
        dg.label_shape
        #[batch_size, train_generator.dim],
        #[batch_size, train_generator.dim],
      )
  )
  return ds

WindowGenerator.make_dataset = make_dataset_water

In [63]:
def plot2(self, model=None, plot_col=0, max_subplots=3, plot_out_col=0):
  inputs, labels = self.example
  plt.figure(figsize=(10, 8))
  plot_col_index = self.column_indices[plot_col]
  plot_out_col_index = self.column_indices[plot_out_col]
  max_n = min(max_subplots, len(inputs))
  for n in range(max_n):
    plt.subplot(3, 1, n+1)
    plt.ylabel(f'{plot_col} [normed]', fontproperties=fprop)
    plt.plot(self.input_indices, inputs[n, :, plot_col_index],
             label='Inputs', marker='.', zorder=-10)

    if self.label_columns:
      label_col_index = self.label_columns_indices.get(plot_col, None)
      label_out_col_index = self.label_columns_indices.get(plot_out_col, None)
    else:
      label_col_index = plot_col_index
      label_out_col_index = plot_out_col_index

    if label_col_index is None:
      continue

    plt.scatter(self.label_indices, labels[n, :, label_out_col_index],
                label='Labels', c='#2ca02c')
    if model is not None:
      predictions = model(inputs)
      plt.scatter(self.label_indices, predictions[n, :, label_out_col_index],
                  marker=None, label='Predictions',
                  c='#ff7f0e')

    if n == 0:
      plt.legend()

  plt.xlabel('Time [h]')

WindowGenerator.plot2 = plot2

**1일단위로 변환**

In [64]:
def hour_to_day_mean(array):
    
#     print("1")
#     print(array)
    
    time = 24
    array = np.reshape(array, (array.shape[0], array.shape[1]//time, time, array.shape[2]))

    array = np.mean(array, axis=2)
    
#     print(array)

    return array

def plot24(self, model=None, plot_col=0, max_subplots=3, plot_out_col=0):
    
  inputs, labels = self.example
  plt.figure(figsize=(10, 8))
  plot_col_index = self.column_indices[plot_col]
  plot_out_col_index = self.column_indices[plot_out_col]
  max_n = min(max_subplots, len(inputs))

  if self.label_columns:
    label_col_index = self.label_columns_indices.get(plot_col, None)
    label_out_col_index = self.label_columns_indices.get(plot_out_col, None)
  else:
    label_col_index = plot_col_index
    label_out_col_index = plot_out_col_index
    
  for n in range(max_n):
    plt.subplot(3, 1, n+1)
    plt.ylabel(f'{df_all.columns[plot_col]} [normed]', fontproperties=fprop)
    
    input_temp = hour_to_day_mean(inputs[n, :, plot_col_index])
    input_temp = (input_temp * (train_std[plot_col] + 1e-6)) + train_mean[plot_col]
#     input_temp = input_temp * train_std[plot_col] + train_mean[plot_col]
    
    plt.plot(
        hour_to_day_mean(self.input_indices),
        input_temp,
        label='Inputs', marker='.', zorder=-10)

    if label_col_index is None:
      continue
    
    label_temp = hour_to_day_mean(labels[n, :, label_out_col_index])
    label_temp = (label_temp * (train_std[plot_col] + 1e-6)) + train_mean[plot_col]
#     label_temp = label_temp * train_std[plot_col] + train_mean[plot_col]
    
    plt.plot(
        hour_to_day_mean(self.label_indices),
        label_temp,
        label='Labels', marker='.', zorder=-10, c='#2ca02c')
    
    if model is not None:
      predictions = model(inputs)
    
      #predictions = predictions * train_std[plot_col] * train_mean[plot_col]
      predict_temp = hour_to_day_mean(predictions[n,:, label_out_col_index])
      predict_temp = (predict_temp * (train_std[plot_col] + 1e-6)) + train_mean[plot_col]
#       predict_temp = predict_temp * train_std[plot_col] + train_mean[plot_col]
    
      plt.plot(
          hour_to_day_mean(self.label_indices),
          predict_temp,
          label='Predictions', marker='.', zorder=-10, c='#ff7f0e')

    if n == 0:
      plt.legend()

  plt.xlabel('Time [day]')

WindowGenerator.plot24 = plot24

**예측률 계산**

In [65]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [112]:
def compa(self, model=None, plot_col=0, max_subplots=3, plot_out_col=0, windows=None):
       
    if example is not None :
        inputs, labels = windows
    else :
        inputs, labels = self.example
        
        
#     plot_out_col_index = self.column_indices[plot_out_col]
    
#     if self.label_columns:
#         label_out_col_index = self.label_columns_indices.get(plot_out_col, None)
#     else:
#         label_out_col_index = plot_out_col_index
    
    if model is None:
        return
    
#     print(plot_col)
    
#     print("column : " + df_all.columns[plot_col])
    
    mae = 0
    mse = 0
    rmse = 0
    mape = 0
    pred_arr = []
    label_arr = []
    
    sum_pred = 0
    sum_label =0
    
    o = 0
    p = 0
    
#     print(train_std[plot_col])
    
#     print(inputs.shape)
#     print(inputs[0].shape)

#     inputs = inputs[0:1]
    
# #     print(inputs)
    
#     labels = labels[0:1]
    
    predictions = model(inputs)
    
    pred = predictions.numpy()
    label = labels.numpy()
    
    for i in range(5) :
#         print(print("column : " + df_all.columns[plot_col[i]]))
        pred[:,:,i] = pred[:,:,i] * train_std[plot_col[i]] + train_mean[plot_col[i]]
        label[:,:,i] = label[:,:,i]* train_std[plot_col[i]] + train_mean[plot_col[i]]

        
#     pred_temp = hour_to_day_mean(pred[127:128,:,:]) 
    #for n in range(len(inputs)):
    #     for i in range(5) :         
     #           pred_temp = hour_to_day_mean(pred[n:n+1,:,i:i+1])
     #           label_temp = hour_to_day_mean(label[n:n+1,:,i:i+1])
    
     #for n in range(len(inputs)):           
      #      pred_temp = hour_to_day_mean(pred[n:n+1,:,:])
      #      label_temp = hour_to_day_mean(label[n:n+1,:,:])
            
#     print(pred.shape)
    
    pred = hour_to_day_mean(pred)
    label = hour_to_day_mean(label)
    
    o = np.sum(label[:,4,:],axis=0)
    
    p = np.sum(pred[:,4,:],axis=0)
    
    print("p")
    print(p)
    print("o")
    print(o)
    
       
    err = np.abs(o-p) / o *100
    
#     print(err.shape)
    
#     print(err)
    
    print(err[0])
    print(err[1])
    print(err[2])
    print(err[3])
    print(err[4])
    
    print("MAE")
    print(train_mean[plot_col[0]])
    print(train_mean[plot_col[1]])
    print(train_mean[plot_col[2]])
    print(train_mean[plot_col[3]])
    print(train_mean[plot_col[4]])
    print(err[0] * train_mean[plot_col[0]])
    print(err[1] * train_mean[plot_col[1]])
    print(err[2] * train_mean[plot_col[2]])
    print(err[3] * train_mean[plot_col[3]])
    print(err[4] * train_mean[plot_col[4]])
    
#     print(np.sum(err,axis=0))
    
    
        
        
#     print(pred_temp)
#     np.sum(pred_temp)
#     print(pred_temp.shape)
    
#     label_temp = hour_to_day_mean(label[0][:][0])
#     print(pred[0,:,0])
#     print("adsfasdfadsfadsf")
#     print(pred_temp)
#     print(pred[0,0,0])
#     print(pred[0,0,1])
#     print(pred[0,0,2])
#     print(pred[0,0,3])
#     print(pred[0,0,4])
#       pred_temp = hour_to_day_mean(pred[0][:][0]) 
#     label_temp = hour_to_day_mean(label[0][:][0])
        
#     print(pred_temp[4])
#     print(pred_temp[4])
    
#     for n in range(len(inputs)):
#         pred_temp = hour_to_day_mean(pred[n][:][0]) 
#         label_temp = hour_to_day_mean(label[n][:][0])
        
#         print(pred_temp[4])
#         print(pred_temp[4])
    
    
#     predictions[:][:][pred_col_no] = predictions[:][:][pred_col_no] * train_std[plot_col] + train_mean[plot_col]
#     labels = labels * train_std[plot_col] + train_mean[plot_col]
    
    
#     print(predictions[0])
# #     print(plot_col)
# #     print(plot_out_col_index)
# #     pred = predictions.numpy()
# #     print(pred.shape)
# #     print(pred)
# #     print(pred[0][0][pred_col_no])

#     label = labels.numpy()
#     print(label.shape)
#     print(label)
#     print(label[0][0][pred_col_no])
    
    
    
#     print('++++++++++++++++++++++++')
#     print(np.sum(predictions.numpy()))
    
#     print(predictions.numpy().sum())
    
#     print('++++++++++++++++++++++++')
#     print(np.sum(labels.numpy()))
    
#     o = np.sum(labels.numpy())
#     p = np.sum(predictions.numpy())
#     print('++++++++++++++++++++++++')
#     print(np.abs(o-p) / o *100)

    
    
    
#     if min_max_normailze :
#         predictions = predictions * train_std[plot_col] + train_mean[plot_col]
#         labels = labels * train_std[plot_col] + train_mean[plot_col]
        
#         predictions = predictions.numpy()
#         labels = labels.numpy()
        
#         predictions = (predictions - predictions.min()) / (predictions.max() - predictions.min())
#         labels = (labels - labels.min()) / (labels.max() - labels.min())
    
#     for n in range(len(inputs)):
#         pred_temp = hour_to_day_mean(predictions[n,:,label_out_col_index]) 
#         label_temp = hour_to_day_mean(labels[n,:,label_out_col_index])
        
#         pred_arr.append(pred_temp[4])
#         label_arr.append(label_temp[4])
        
# #         print(pred_temp[4].numpy())

        
#         sum_pred = sum_pred + pred_temp[4].numpy()
#         sum_label = sum_label + label_temp[4].numpy()
       
        
#     mae = mean_absolute_error(label_arr, pred_arr) 
#     mse = mean_squared_error(label_arr, pred_arr)
#     rmse = np.sqrt(mse)
#     mape = np.mean(np.abs((np.array(label_arr) - np.array(pred_arr))/np.array(label_arr)) )*100
    
   

# #     print(sum_pred)
#     pbias = ( np.abs(sum_label - sum_pred) / sum_label ) * 100 
    
# #     pbias = ( (sum_label - sum_pred) / sum_label ) * 100 
    
#     print("mae:")  
#     print(mae)
    
#     print("mse:")  
#     print(mse)
    
#     print("rmse")  
#     print(rmse)
    
#     print("mape")  
#     print(mape)
    
#     print('pbias')
#     print(pbias)
    
    

WindowGenerator.compa = compa

In [67]:
np.abs(5.9624996  - 8.015346 ) / 5.9624996 *100

34.42929203718519

In [68]:
30 * 128

3840

In [69]:
train_std

수온           8.616625
수소이온농도       0.765725
전기전도도       89.425099
용존산소         3.681855
총유기탄소        1.151644
총질소          1.781291
총인           0.047667
클로로필-a      38.660971
Day sin      0.707109
Day cos      0.707109
Year sin     0.707102
Year cos     0.707117
dtype: float64

In [70]:
train_df.shape

(28032, 36)

In [71]:
#multi_window.compa(elman_model, plot_col=out_features[0])

In [72]:
# target_col = '용존산소'
# num_features = df[0].shape[1]
# label_columns_indices = {name: i for i, name in enumerate(df[0])}
# target_col_idx = label_columns_indices[target_col]
# # target_col_idx
# out_features = [target_col_idx]
# # out_features = [3,5,6,7,8]
# out_num_features = len(out_features)
# out_num_features

In [73]:
def compile_and_fit(model, window, patience=1000):
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    mode='min')

  model.compile(loss=tf.losses.MeanSquaredError(),
                optimizer=tf.optimizers.Adam(),
                metrics=[tf.metrics.MeanAbsoluteError()])
#   model.compile(loss=tf.losses.MeanSquaredError(),
#                 optimizer=tf.optimizers.Adam(),
#                 metrics=[tf.metrics.MeanSquaredError()])
  #model.compile(loss=GAIN.RMSE_loss)

  history = model.fit(window.train, epochs=MAX_EPOCHS,
                      validation_data=window.val,
                      callbacks=[early_stopping])
  return history

In [74]:
def plot_history(history):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.plot(history.history['loss'], label='loss')
    ax.plot(history.history['mean_absolute_error'], label='mae')
#     ax.plot(history.history['mean_squared_error'], label='mse')
    ax.plot(history.history['val_loss'], label='val_loss')
    ax.plot(history.history['val_mean_absolute_error'], label='val_mae')
#     ax.plot(history.history['val_mean_squared_error'], label='val_mse')
    #plt.legend(history.history.keys(), loc='upper right')
    #ax.legend(loc='upper center')
    ax.legend()
    ax.set_xlabel("epochs")
    ax.set_ylabel("loss")
    plt.show()

In [75]:
multi_window = WindowGenerator(input_width=24*7,
                               label_width=OUT_STEPS,
                               shift=OUT_STEPS,
                               train_df=train_df,
                               val_df=val_df,
                               test_df=test_df,
                               out_features = out_features,
                               out_num_features = out_num_features,
                               )

In [76]:
multi_val_performance = {}
multi_performance = {}

**Elman_RNN**

In [77]:
elman_model = tf.keras.Sequential([
    # Take the last time step.
    # Shape [batch, time, features] => [batch, 1, features]
    tf.keras.layers.SimpleRNN(128, return_sequences=False),

    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),

    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

In [78]:
elman_model = tf.keras.Sequential([
    # Take the last time step.
    # Shape [batch, time, features] => [batch, 1, features]
#     tf.keras.layers.SimpleRNN(128, return_sequences=True),
#     tf.keras.layers.SimpleRNN(128, return_sequences=True),
#     tf.keras.layers.SimpleRNN(128, return_sequences=True),
#     tf.keras.layers.SimpleRNN(128, return_sequences=True),
#     tf.keras.layers.SimpleRNN(128, return_sequences=True),
#     tf.keras.layers.SimpleRNN(128, return_sequences=True),
#     tf.keras.layers.SimpleRNN(128, return_sequences=True),
#     tf.keras.layers.SimpleRNN(128, return_sequences=True),
    tf.keras.layers.SimpleRNN(128, return_sequences=False),

    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),

    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

In [79]:
history = compile_and_fit(elman_model, multi_window)
multi_val_performance['ELMAN_RNN'] = elman_model.evaluate(multi_window.val.repeat(-1), steps=100)
multi_performance['ELMAN_RNN'] = elman_model.evaluate(multi_window.test.repeat(-1), verbose=1, steps=100)
print('val performance =', multi_val_performance['ELMAN_RNN'])
print('test performance = ', multi_performance['ELMAN_RNN'])

Epoch 1/150
1/1 [==============================] - 1s 808ms/step - loss: 1.1897 - mean_absolute_error: 0.7839 - val_loss: 0.9956 - val_mean_absolute_error: 0.7604
Epoch 2/150
1/1 [==============================] - 0s 276ms/step - loss: 1.2067 - mean_absolute_error: 0.7911 - val_loss: 0.9768 - val_mean_absolute_error: 0.7430
Epoch 3/150
1/1 [==============================] - 0s 274ms/step - loss: 1.0728 - mean_absolute_error: 0.7422 - val_loss: 0.8246 - val_mean_absolute_error: 0.6740
Epoch 4/150
1/1 [==============================] - 0s 285ms/step - loss: 1.0109 - mean_absolute_error: 0.7165 - val_loss: 0.9467 - val_mean_absolute_error: 0.7331
Epoch 5/150
1/1 [==============================] - 0s 259ms/step - loss: 1.1115 - mean_absolute_error: 0.7482 - val_loss: 0.8732 - val_mean_absolute_error: 0.6912
Epoch 6/150
1/1 [==============================] - 0s 280ms/step - loss: 0.9895 - mean_absolute_error: 0.6988 - val_loss: 0.8584 - val_mean_absolute_error: 0.6756
Epoch 7/150
1/1 [=====

1/1 [==============================] - 0s 271ms/step - loss: 0.3641 - mean_absolute_error: 0.4343 - val_loss: 0.4924 - val_mean_absolute_error: 0.5195
Epoch 52/150
1/1 [==============================] - 0s 300ms/step - loss: 0.3757 - mean_absolute_error: 0.4384 - val_loss: 0.4770 - val_mean_absolute_error: 0.5150
Epoch 53/150
1/1 [==============================] - 0s 277ms/step - loss: 0.3773 - mean_absolute_error: 0.4370 - val_loss: 0.4379 - val_mean_absolute_error: 0.4955
Epoch 54/150
1/1 [==============================] - 0s 295ms/step - loss: 0.3917 - mean_absolute_error: 0.4469 - val_loss: 0.4351 - val_mean_absolute_error: 0.4955
Epoch 55/150
1/1 [==============================] - 0s 283ms/step - loss: 0.4070 - mean_absolute_error: 0.4563 - val_loss: 0.4715 - val_mean_absolute_error: 0.5046
Epoch 56/150
1/1 [==============================] - 0s 292ms/step - loss: 0.3492 - mean_absolute_error: 0.4158 - val_loss: 0.4266 - val_mean_absolute_error: 0.4832
Epoch 57/150
1/1 [===========

1/1 [==============================] - 0s 269ms/step - loss: 0.2886 - mean_absolute_error: 0.3721 - val_loss: 0.4637 - val_mean_absolute_error: 0.4960
Epoch 102/150
1/1 [==============================] - 0s 267ms/step - loss: 0.2918 - mean_absolute_error: 0.3804 - val_loss: 0.4842 - val_mean_absolute_error: 0.5079
Epoch 103/150
1/1 [==============================] - 0s 335ms/step - loss: 0.2831 - mean_absolute_error: 0.3705 - val_loss: 0.4139 - val_mean_absolute_error: 0.4693
Epoch 104/150
1/1 [==============================] - 0s 332ms/step - loss: 0.2596 - mean_absolute_error: 0.3549 - val_loss: 0.4056 - val_mean_absolute_error: 0.4663
Epoch 105/150
1/1 [==============================] - 0s 289ms/step - loss: 0.2835 - mean_absolute_error: 0.3720 - val_loss: 0.4467 - val_mean_absolute_error: 0.4832
Epoch 106/150
1/1 [==============================] - 0s 275ms/step - loss: 0.3051 - mean_absolute_error: 0.3734 - val_loss: 0.4248 - val_mean_absolute_error: 0.4731
Epoch 107/150
1/1 [=====

100/100 [==============================] - 9s 94ms/step - loss: 0.3375 - mean_absolute_error: 0.4441
val performance = [0.45088493824005127, 0.48916715383529663]
test performance =  [0.3375037908554077, 0.44412773847579956]


In [80]:
plot_history(history)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [81]:
multi_window.plot2(elman_model, plot_col=out_features[0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**GRU**

In [82]:
gru_model = tf.keras.Sequential([
    # Take the last time step.
    # Shape [batch, time, features] => [batch, 1, features]
#     tf.keras.layers.GRU(128, return_sequences=True),
#     tf.keras.layers.GRU(128, return_sequences=True),
#     tf.keras.layers.GRU(128, return_sequences=True),
#     tf.keras.layers.GRU(128, return_sequences=True),
#     tf.keras.layers.GRU(128, return_sequences=True),
#     tf.keras.layers.GRU(128, return_sequences=True),
#     tf.keras.layers.GRU(128, return_sequences=True),
#     tf.keras.layers.GRU(128, return_sequences=True),
    tf.keras.layers.GRU(128, return_sequences=False),
    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

In [83]:
history = compile_and_fit(gru_model, multi_window)
multi_val_performance['GRU'] = gru_model.evaluate(multi_window.val.repeat(-1), steps=100)
multi_performance['GRU'] = gru_model.evaluate(multi_window.test.repeat(-1), verbose=1, steps=100)
print('val performance =', multi_val_performance['GRU'])
print('test performance = ', multi_performance['GRU'])

Epoch 1/150
1/1 [==============================] - 1s 924ms/step - loss: 1.1727 - mean_absolute_error: 0.7608 - val_loss: 0.9865 - val_mean_absolute_error: 0.7484
Epoch 2/150
1/1 [==============================] - 0s 365ms/step - loss: 1.3362 - mean_absolute_error: 0.8050 - val_loss: 0.9831 - val_mean_absolute_error: 0.7465
Epoch 3/150
1/1 [==============================] - 0s 353ms/step - loss: 1.0106 - mean_absolute_error: 0.7399 - val_loss: 0.9843 - val_mean_absolute_error: 0.7511
Epoch 4/150
1/1 [==============================] - 0s 307ms/step - loss: 1.0093 - mean_absolute_error: 0.7041 - val_loss: 0.9532 - val_mean_absolute_error: 0.7447
Epoch 5/150
1/1 [==============================] - 0s 324ms/step - loss: 1.1619 - mean_absolute_error: 0.7619 - val_loss: 0.9124 - val_mean_absolute_error: 0.7257
Epoch 6/150
1/1 [==============================] - 0s 313ms/step - loss: 1.0187 - mean_absolute_error: 0.7070 - val_loss: 0.8689 - val_mean_absolute_error: 0.7006
Epoch 7/150
1/1 [=====

1/1 [==============================] - 0s 343ms/step - loss: 0.3834 - mean_absolute_error: 0.4416 - val_loss: 0.5256 - val_mean_absolute_error: 0.5251
Epoch 52/150
1/1 [==============================] - 0s 324ms/step - loss: 0.4342 - mean_absolute_error: 0.4781 - val_loss: 0.5351 - val_mean_absolute_error: 0.5388
Epoch 53/150
1/1 [==============================] - 0s 307ms/step - loss: 0.3467 - mean_absolute_error: 0.4371 - val_loss: 0.5128 - val_mean_absolute_error: 0.5198
Epoch 54/150
1/1 [==============================] - 0s 324ms/step - loss: 0.3956 - mean_absolute_error: 0.4508 - val_loss: 0.5002 - val_mean_absolute_error: 0.5263
Epoch 55/150
1/1 [==============================] - 0s 320ms/step - loss: 0.4046 - mean_absolute_error: 0.4651 - val_loss: 0.4675 - val_mean_absolute_error: 0.5053
Epoch 56/150
1/1 [==============================] - 0s 324ms/step - loss: 0.3695 - mean_absolute_error: 0.4394 - val_loss: 0.5311 - val_mean_absolute_error: 0.5347
Epoch 57/150
1/1 [===========

1/1 [==============================] - 0s 340ms/step - loss: 0.3208 - mean_absolute_error: 0.3977 - val_loss: 0.4522 - val_mean_absolute_error: 0.4777
Epoch 102/150
1/1 [==============================] - 0s 331ms/step - loss: 0.3463 - mean_absolute_error: 0.4073 - val_loss: 0.4666 - val_mean_absolute_error: 0.4919
Epoch 103/150
1/1 [==============================] - 0s 326ms/step - loss: 0.3271 - mean_absolute_error: 0.3973 - val_loss: 0.4332 - val_mean_absolute_error: 0.4712
Epoch 104/150
1/1 [==============================] - 0s 324ms/step - loss: 0.3605 - mean_absolute_error: 0.4108 - val_loss: 0.4738 - val_mean_absolute_error: 0.4994
Epoch 105/150
1/1 [==============================] - 0s 320ms/step - loss: 0.3059 - mean_absolute_error: 0.3883 - val_loss: 0.4187 - val_mean_absolute_error: 0.4709
Epoch 106/150
1/1 [==============================] - 0s 314ms/step - loss: 0.3702 - mean_absolute_error: 0.4072 - val_loss: 0.4661 - val_mean_absolute_error: 0.4920
Epoch 107/150
1/1 [=====

100/100 [==============================] - 10s 99ms/step - loss: 0.3194 - mean_absolute_error: 0.4213
val performance = [0.40422946214675903, 0.44964879751205444]
test performance =  [0.31941062211990356, 0.4212551712989807]


**LSTM**

In [84]:
multi_lstm_model = tf.keras.Sequential([
    # Shape [batch, time, features] => [batch, lstm_units]
    # Adding more `lstm_units` just overfits more quickly.
    #tf.keras.layers.LSTM(32, return_sequences=False),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
    tf.keras.layers.LSTM(128, return_sequences=False),
    # Shape => [batch, out_steps*features]
    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

In [85]:
history = compile_and_fit(multi_lstm_model, multi_window)

#IPython.display.clear_output()

multi_val_performance['LSTM'] = multi_lstm_model.evaluate(multi_window.val)
multi_performance['LSTM'] = multi_lstm_model.evaluate(multi_window.test, verbose=0)
print('val performance =', multi_val_performance['LSTM'])
print('test performance = ', multi_performance['LSTM'])

Epoch 1/150
1/1 [==============================] - 2s 2s/step - loss: 1.1225 - mean_absolute_error: 0.7526 - val_loss: 0.9712 - val_mean_absolute_error: 0.7527
Epoch 2/150
1/1 [==============================] - 0s 270ms/step - loss: 1.0312 - mean_absolute_error: 0.7470 - val_loss: 0.9499 - val_mean_absolute_error: 0.7385
Epoch 3/150
1/1 [==============================] - 0s 261ms/step - loss: 1.0773 - mean_absolute_error: 0.7299 - val_loss: 1.0099 - val_mean_absolute_error: 0.7587
Epoch 4/150
1/1 [==============================] - 0s 367ms/step - loss: 0.9553 - mean_absolute_error: 0.6871 - val_loss: 0.8704 - val_mean_absolute_error: 0.6999
Epoch 5/150
1/1 [==============================] - 0s 251ms/step - loss: 1.0304 - mean_absolute_error: 0.6994 - val_loss: 0.8467 - val_mean_absolute_error: 0.6950
Epoch 6/150
1/1 [==============================] - 0s 279ms/step - loss: 0.9535 - mean_absolute_error: 0.6965 - val_loss: 0.9486 - val_mean_absolute_error: 0.7336
Epoch 7/150
1/1 [========

1/1 [==============================] - 0s 258ms/step - loss: 0.3321 - mean_absolute_error: 0.4203 - val_loss: 0.5082 - val_mean_absolute_error: 0.5395
Epoch 52/150
1/1 [==============================] - 0s 275ms/step - loss: 0.3199 - mean_absolute_error: 0.4138 - val_loss: 0.4834 - val_mean_absolute_error: 0.5202
Epoch 53/150
1/1 [==============================] - 0s 260ms/step - loss: 0.3375 - mean_absolute_error: 0.4274 - val_loss: 0.4659 - val_mean_absolute_error: 0.5187
Epoch 54/150
1/1 [==============================] - 0s 266ms/step - loss: 0.3740 - mean_absolute_error: 0.4422 - val_loss: 0.4563 - val_mean_absolute_error: 0.5125
Epoch 55/150
1/1 [==============================] - 0s 294ms/step - loss: 0.3602 - mean_absolute_error: 0.4354 - val_loss: 0.4197 - val_mean_absolute_error: 0.4863
Epoch 56/150
1/1 [==============================] - 0s 280ms/step - loss: 0.4183 - mean_absolute_error: 0.4522 - val_loss: 0.4718 - val_mean_absolute_error: 0.5117
Epoch 57/150
1/1 [===========

1/1 [==============================] - 0s 281ms/step - loss: 0.2398 - mean_absolute_error: 0.3527 - val_loss: 0.4305 - val_mean_absolute_error: 0.4883
Epoch 102/150
1/1 [==============================] - 0s 298ms/step - loss: 0.2658 - mean_absolute_error: 0.3657 - val_loss: 0.4225 - val_mean_absolute_error: 0.4822
Epoch 103/150
1/1 [==============================] - 0s 264ms/step - loss: 0.2769 - mean_absolute_error: 0.3698 - val_loss: 0.4369 - val_mean_absolute_error: 0.4889
Epoch 104/150
1/1 [==============================] - 0s 278ms/step - loss: 0.2673 - mean_absolute_error: 0.3714 - val_loss: 0.4527 - val_mean_absolute_error: 0.5025
Epoch 105/150
1/1 [==============================] - 0s 258ms/step - loss: 0.3001 - mean_absolute_error: 0.3808 - val_loss: 0.3995 - val_mean_absolute_error: 0.4628
Epoch 106/150
1/1 [==============================] - 0s 228ms/step - loss: 0.2685 - mean_absolute_error: 0.3624 - val_loss: 0.4013 - val_mean_absolute_error: 0.4681
Epoch 107/150
1/1 [=====

1/1 [==============================] - 0s 123ms/step - loss: 0.4765 - mean_absolute_error: 0.5150
val performance = [0.47647425532341003, 0.5150293707847595]
test performance =  [0.44634583592414856, 0.5051835179328918]


**CNN model**

In [86]:
CONV_WIDTH = 7
CONV_LAYER_NO = 1
multi_conv_model = tf.keras.Sequential([
    # Shape [batch, time, features] => [batch, CONV_WIDTH, features]
    tf.keras.layers.Lambda(lambda x: x[:, -(CONV_WIDTH*CONV_LAYER_NO-CONV_LAYER_NO+1):, :]),
] + [
    # Shape => [batch, 1, conv_units]
    tf.keras.layers.Conv1D(1024, activation='relu', kernel_size=(CONV_WIDTH)) for i in range(CONV_LAYER_NO)
] + [
    # Shape => [batch, 1,  out_steps*features]
    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

In [87]:
history = compile_and_fit(multi_conv_model, multi_window)

#IPython.display.clear_output()

multi_val_performance['Conv'] = multi_conv_model.evaluate(multi_window.val)
multi_performance['Conv'] = multi_conv_model.evaluate(multi_window.test, verbose=0)
print('val performance =', multi_val_performance['Conv'])
print('test performance = ', multi_performance['Conv'])

Epoch 1/150
1/1 [==============================] - 1s 733ms/step - loss: 1.3093 - mean_absolute_error: 0.8256 - val_loss: 0.8744 - val_mean_absolute_error: 0.7173
Epoch 2/150
1/1 [==============================] - 0s 235ms/step - loss: 1.0905 - mean_absolute_error: 0.7451 - val_loss: 0.8886 - val_mean_absolute_error: 0.7172
Epoch 3/150
1/1 [==============================] - 0s 215ms/step - loss: 0.9600 - mean_absolute_error: 0.6813 - val_loss: 0.8405 - val_mean_absolute_error: 0.7087
Epoch 4/150
1/1 [==============================] - 0s 265ms/step - loss: 0.7542 - mean_absolute_error: 0.6462 - val_loss: 0.7769 - val_mean_absolute_error: 0.6730
Epoch 5/150
1/1 [==============================] - 0s 242ms/step - loss: 0.6676 - mean_absolute_error: 0.6329 - val_loss: 0.7786 - val_mean_absolute_error: 0.6925
Epoch 6/150
1/1 [==============================] - 0s 257ms/step - loss: 0.6161 - mean_absolute_error: 0.6163 - val_loss: 0.8547 - val_mean_absolute_error: 0.7264
Epoch 7/150
1/1 [=====

1/1 [==============================] - 0s 255ms/step - loss: 0.2873 - mean_absolute_error: 0.3821 - val_loss: 0.4178 - val_mean_absolute_error: 0.4640
Epoch 52/150
1/1 [==============================] - 0s 254ms/step - loss: 0.2371 - mean_absolute_error: 0.3474 - val_loss: 0.4234 - val_mean_absolute_error: 0.4642
Epoch 53/150
1/1 [==============================] - 0s 242ms/step - loss: 0.3492 - mean_absolute_error: 0.3987 - val_loss: 0.4290 - val_mean_absolute_error: 0.4713
Epoch 54/150
1/1 [==============================] - 0s 265ms/step - loss: 0.2639 - mean_absolute_error: 0.3556 - val_loss: 0.4304 - val_mean_absolute_error: 0.4751
Epoch 55/150
1/1 [==============================] - 0s 253ms/step - loss: 0.3218 - mean_absolute_error: 0.3799 - val_loss: 0.4459 - val_mean_absolute_error: 0.4818
Epoch 56/150
1/1 [==============================] - 0s 260ms/step - loss: 0.2824 - mean_absolute_error: 0.3680 - val_loss: 0.4668 - val_mean_absolute_error: 0.4940
Epoch 57/150
1/1 [===========

1/1 [==============================] - 0s 258ms/step - loss: 0.2533 - mean_absolute_error: 0.3493 - val_loss: 0.4698 - val_mean_absolute_error: 0.4989
Epoch 102/150
1/1 [==============================] - 0s 257ms/step - loss: 0.2688 - mean_absolute_error: 0.3669 - val_loss: 0.4508 - val_mean_absolute_error: 0.4803
Epoch 103/150
1/1 [==============================] - 0s 229ms/step - loss: 0.2404 - mean_absolute_error: 0.3402 - val_loss: 0.4551 - val_mean_absolute_error: 0.4885
Epoch 104/150
1/1 [==============================] - 0s 290ms/step - loss: 0.2726 - mean_absolute_error: 0.3564 - val_loss: 0.4797 - val_mean_absolute_error: 0.5079
Epoch 105/150
1/1 [==============================] - 0s 278ms/step - loss: 0.2501 - mean_absolute_error: 0.3369 - val_loss: 0.4800 - val_mean_absolute_error: 0.5038
Epoch 106/150
1/1 [==============================] - 0s 244ms/step - loss: 0.2897 - mean_absolute_error: 0.3508 - val_loss: 0.4574 - val_mean_absolute_error: 0.4894
Epoch 107/150
1/1 [=====

1/1 [==============================] - 0s 123ms/step - loss: 0.4824 - mean_absolute_error: 0.5027
val performance = [0.48238709568977356, 0.5026800036430359]
test performance =  [0.3602927625179291, 0.45417794585227966]


In [88]:
multi_val_performance['Conv'] = multi_conv_model.evaluate(multi_window.val)
multi_performance['Conv'] = multi_conv_model.evaluate(multi_window.test, verbose=0)
print('val performance =', multi_val_performance['Conv'])
print('test performance = ', multi_performance['Conv'])

1/1 [==============================] - 0s 125ms/step - loss: 0.4367 - mean_absolute_error: 0.4804
val performance = [0.43669846653938293, 0.48044103384017944]
test performance =  [0.38824528455734253, 0.45713385939598083]


**performance**

In [89]:
x = np.arange(len(multi_performance))
width = 0.3


metric_name = 'mean_absolute_error'
metric_index = multi_conv_model.metrics_names.index('mean_absolute_error')
val_mae = [v[metric_index] for v in multi_val_performance.values()]
test_mae = [v[metric_index] for v in multi_performance.values()]

plt.figure()
plt.bar(x - 0.17, val_mae, width, label='Validation')
plt.bar(x + 0.17, test_mae, width, label='Test')
plt.xticks(ticks=x, labels=multi_performance.keys(),
           rotation=45)
plt.ylabel(f'MAE (average over all times and outputs)')
_ = plt.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [124]:
multi_window.compa(gru_model, plot_col=out_features, windows=multi_window.example3)

p
[1055.4779    470.31708   516.8782     11.953159 4653.068   ]
o
[1325.8617    465.33945   497.33432     8.959917 4125.2925  ]
20.393063
1.0696774
3.9297216
33.40703
12.793648
MAE
8.789810443490579
3.848232051049673
3.5566499402851033
0.09112342023943747
36.92030248143427
179.25115454198726
4.1163666737320455
13.97664407212791
3.04416266945916
472.34534536582817


In [121]:
train_mean

수온          1.754832e+01
수소이온농도      7.404510e+00
전기전도도       2.903623e+02
용존산소        8.789810e+00
총유기탄소       3.848232e+00
총질소         3.556650e+00
총인          9.112342e-02
클로로필-a      3.692030e+01
Day sin     6.037102e-15
Day cos     2.747704e-15
Year sin   -5.681266e-08
Year cos    2.053380e-05
dtype: float64

In [111]:
8.134553 * 0.3692

3.0032769675999997